In [ ]:
import sys
from PyQt5 import uic, QtGui,QtCore
from PyQt5.QtGui import QImage,QPixmap
from PyQt5.QtWidgets import QWidget, QFrame, QMainWindow, QApplication, QFileDialog, QVBoxLayout
#sys.path.append(r"C:\Users\jrosell\")
from HyperspectralJRM import HypSpImage
import numpy as np
import random
import cv2
from matplotlib import colors
import os
import glob
import re
from datetime import datetime
import math
import csv

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg 
from matplotlib.figure import Figure

import HyperspectralPatterns as hp


class MatplotlibWidget(QWidget):
    def __init__(self, parent=None):
        super(MatplotlibWidget, self).__init__(parent)
        self.figure = Figure()
        self.canvas = FigureCanvasQTAgg(self.figure)
        self.axis = self.figure.add_subplot(111)
        self.figure.tight_layout()
        self.layoutvertical = QVBoxLayout(self)
        self.layoutvertical.setContentsMargins(0,0,0,0)
        self.layoutvertical.addWidget(self.canvas)
           

#################################################################################################################    
## Hyperspectral_GUI: This class corresponds to the one that manages the user interface 
#################################################################################################################    

def analizeBlobs(img1,img2, img3m):
    
    return 
                 

class Hyperspectral_GUI(QMainWindow):
    
    def __init__(self):
        super().__init__()

        # Default values
        self._IR_WIDTH = 320
        self._IR_HEIGHT = 32
        self._HSC_WIDTH = 256 
        self._HSC_HEIGHT = 32
        
        self._MIN_BLOB = 30
        
        self._OffX_HSCvsIR = 80
        self._OffY_HSCvsIR = 3
        
        self._WAVELENGTH_RED = 685 
        self._WAVELENGTH_GREEN = 535
        self._WAVELENGTH_BLUE = 463
        
        self._camHSC = 0
        self._camIR  = 1
        
        self._DO_NOT_SEGMENT = 0
        self._BINARIZE = 1
        self._MARKS = 2
        
        
        #JRM: imatges obtingudes amb _readHypercubes
        self.img_Color_pre = []
        self.img_Color = []
        self.img_Color_post = []
        
        # Pointers to hypersctretral structures of images readed (HSC and IR) 
        self.imHyperHSC_Pre = -1
        self.imHyperHSC = -1
        self.imHyperHSC_Post = -1
        self.imHyperIR_Pre = -1
        self.imHyperIR = -1
        self.imHyperIR_Post = -1
        self.imHyperHSC_Pre_enabled = False
        self.imHyperHSC_enabled = False
        self.imHyperHSC_Post_enabled = False
        self.imHyperIR_Pre_enabled = False
        self.imHyperIR_enabled = False
        self.imHyperIR_Post_enabled = False
        
        # Pointers to hypersctretral structures that contain the interval (Max-Min within each band) corresponding to the baclground image  
        self.imgHyperHSC_BackGround_Max =[]
        self.imgHyperHSC_BackGround_Min =[]
        self.imgHyperIR_BackGround_Max =[]
        self.imgHyperIR_BackGround_Min =[]        
        
        # Link the events of the GUI (buttons, etc, ..) to the functions of the class 'Hyperspectral_GUI'
        uic.loadUi(r"Hyperspectral_GUI_desinger.ui",self)
        self.pButt_FX10File.clicked.connect(self.read_F10_File)
        self.pButt_FX17File.clicked.connect(self.read_F17_File)
        self.pButt_ColorFile.clicked.connect(self.read_Color_Image)
        self.pButt_HSCFile.clicked.connect(self.read_HSC_Image)
        self.pButt_IRFile.clicked.connect(self.read_IR_Image)
        self.pButt_SelectFolder.clicked.connect(self.selectFolder)
        self.pButt_inspectBackground.clicked.connect(self.learningBackground)
        self.pButt_learningPattern.clicked.connect(self.learningPattern)
        self.pButt_batchInspection.clicked.connect(self.batchInspection)
        self.pButt_previousImage.clicked.connect(self.selectedPreviousImage)
        self.pButt_nextImage.clicked.connect(self.selectedNextImage)       
        self.pButt_DeletePattern.clicked.connect(self.deletePattern)    
        self.listWidget.itemDoubleClicked.connect(self.selectedInImageInList)
        self.listWidget.itemClicked.connect(self.selectedInImageInList)
        self.listPatterns.itemDoubleClicked.connect(self.selectedPatternInList)
        self.listPatterns.itemClicked.connect(self.selectedPatternInList)        
        self.cb_MarkBG.stateChanged.connect(self.markBackground)
        self.cb_MarkBlobs.stateChanged.connect(self.markBlobs)
        self.cb_MarkBoundingBox.stateChanged.connect(self.markBoundingBox)

        self.cb_recordSpectrums.stateChanged.connect(self.recordSpectrumsInFile)
        
        self.pb_resetGraph.clicked.connect(self.resetGraph)
        
        self.rb_Segm_Not.clicked.connect(self.doNotSegment)
        self.rb_Segm_Bin.clicked.connect(self.selectBinary)
        self.rb_Segm_Mark.clicked.connect(self.selectMark)
        self.rb_graph_SelectPixel.clicked.connect(self.selectPixel)
        self.rb_graph_SelectBlob.clicked.connect(self.selectBlob)
        
        self.lab_AnalyzingPattern.hide()
        
        # Initialize the values of the components that are shown in the GUI
        self.F10FilePath=""
        self.F17FilePath=""
        self.colorImagePath = ""
        self.HSCImagePath = ""
        self.IRImagePath = ""
        
        self.horSlider_HSC_band1.setMinimum(0)
        self.horSlider_HSC_band1.setMaximum(99)
        self.horSlider_HSC_band1.setSingleStep(1)
        self.horSlider_HSC_band1.setValue(50)
        self.horSlider_HSC_band1.valueChanged.connect(self.getHorSlider_HSC_band1)
    
        self.horSlider_IR_band1.setMinimum(0)
        self.horSlider_IR_band1.setMaximum(99)
        self.horSlider_IR_band1.setSingleStep(1)
        self.horSlider_IR_band1.setValue(50)
        self.horSlider_IR_band1.valueChanged.connect(self.getHorSlider_IR_band1)
        
        # Catch click mouse in images
        self.labHSC.mousePressEvent = self.getPosMouseHSC        
        self.labHSC_pre.mousePressEvent = self.getPosMouseHSC_pre
        self.labHSC_post.mousePressEvent = self.getPosMouseHSC_post
        self.labIR.mousePressEvent = self.getPosMouseIR
        self.labIR_pre.mousePressEvent = self.getPosMouseIR_pre
        self.labIR_post.mousePressEvent = self.getPosMouseIR_post
        
        # MAXIMUM 15 colors 
        self.colors = ['#F58C00','#0055AF','#FA0000','#00FA55','#AF00AF','#AFAF00','#00AF80','#00FAFA','#FF8080','#808000','#8080FA','#008000','#0000FA','#FF80FF','#BB5000']
        self.MaxColor = len(self.colors)    
        self.idxColor = 0

        # 
        self.inspectPixels = [[0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0],
                             [0,0,'',0,0,0,0,0,0,0]]
       
        colorOnlyHSC = 'b'
        colorOnlyIR = 'g'
        colorHSC_IR = '#ffa500'
        
        self.idxChartPixel = 0
        self.idxChartBlob = 0
        
        self.lastsSeriesHSCIR_0= []   #  Conté les 2 últimes inspeccions realitzades
        self.lastsSeriesHSCIR_1= []  #  Conté les 2 últimes inspeccions realitzades
        self.saveSeriesBlobs = False

        self.inspectBlobs = [[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,''],[0,0,'']]

        self.spectrum_HSC_mean = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_lower_00 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_lower_11 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_lower_22 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_lower_33 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_lower_44 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_upper_55 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_upper_66 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_upper_77 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_upper_88 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_HSC_upper_99 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]] 
        self.spectrum_IR_mean = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_lower_00 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_lower_11 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_lower_22 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_lower_33 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_lower_44 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_upper_55 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_upper_66 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_upper_77 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_upper_88 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        self.spectrum_IR_upper_99 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        
        # Index to the currently selected image.
        # it is useful for sequentially traversing a set of images (no need to parse image_Pre, image, and image_post). 
        self.idxPrevious = -1 
              
        self.F10_readed=False
        self.F17_readed=False
        self.folderpath = ''        
        self.listImgs=[]
        self.idxPatterns = -1
        self.idxImgs = -1
        self.totalImgs = 0
        self.checkedBackground = False
        self.init_widgets()

        
        self.imgHSC_Binary = []   # Image binary of blobs (format RGB)
        self.imgHSC_Coloring = [] # Image with labels of blobs (0--> Background)
        self.CurrentImgHSC_Coloring = []
        self.imgIR_Binary = []
        self.imgIR_Coloring = []
        self.CurrentImgIR_Coloring = []
        
        
        # The images shown in main form (after applying the mask). They are useful for optimizing process 
        # time when processing and displaying a set of images sequentially. For each iteration only the two 
        # new images to be displayed (HSC and IR) are processed, not the 6 images displayed in the main form. 
        self.imgScaledpHSC_pre = []
        self.imgScaledHSC = []
        self.imgScaledHSC_post = []
        self.imgScaledIR_pre = []
        self.imgScaledIR = []
        self.imgScaledIR_post = []
        
        self.imgLabelHSC_pre = []
        self.imgLabelHSC = []
        self.imgLabelHSC_post = []
        self.imgLabelIR_pre = []
        self.imgLabelIR = []
        self.imgLabelIR_post = []
        self.labelsHSC_Only = []
        self.labelsIR_Only = []
        self.labelsHSCIR_Match = []
        
        self.imgMaskHSC_pre = []
        self.imgMaskHSC = []
        self.imgMaskHSC_post = []
        self.imgMaskIR_pre = []
        self.imgMaskIR = []
        self.imgMaskIR_post = []    
        
        self.cb_MarkBoundingBox.hide()
        
        self.comboBox_plasticType.addItem("Others")
        self.comboBox_plasticType.addItem("Plastic_01")
        self.comboBox_plasticType.addItem("Plastic_02")
        self.comboBox_plasticType.addItem("Plastic_03")
        self.comboBox_plasticType.addItem("Plastic_04")
        self.comboBox_plasticType.addItem("Plastic_05")
        self.comboBox_plasticType.addItem("Plastic_06")
        self.comboBox_plasticType.addItem("Plastic_07")
        self.comboBox_plasticType.addItem("Plastic_08")
        self.comboBox_plasticType.addItem("Plastic_09")
        self.comboBox_plasticType.addItem("Plastic_10")
        
        self.idxPixel_findPattern = 0
        self.idxImage_findPattern = 0
            
        self.lab_RequiresMessage_2.setHidden(True)
        
        self.label_Aux1.setText("")
        self.label_Aux2.setText("")
        self.label_Aux3.setText("")
        
        for file in glob.glob(r".\Patterns\Patt*.csv"):
            self.listPatterns.addItem(os.path.splitext(os.path.basename(file))[0])

    def init_widgets(self):
        self.matplotlibwidget_HSC = MatplotlibWidget()
        self.layoutvertical_HSC = QVBoxLayout(self.widget_HSC)
        self.layoutvertical_HSC.addWidget(self.matplotlibwidget_HSC)
        
        self.matplotlibwidget_IR = MatplotlibWidget()
        self.layoutvertical_IR = QVBoxLayout(self.widget_IR)
        self.layoutvertical_IR.addWidget(self.matplotlibwidget_IR)
################################################################################################        
## matchLabels --> Find labels of 3 HSC gray images and 3 IR gray images, and match blobs between then.
## -- Inputs -----------------------------------------------------------------------------------
## grayHSC0,grayHSC1,grayHSC2 -->
## grayIR0,grayIR1,grayIR2 -->
## offsetX,offsetY -->
## -- Outputs ----------------------------------------------------------------------------------
## Match --> list of labels matched between HSC and IR. ex.[[labHSC1,labIR1],[labHSC2,labIR2],...]
## NoMatch1,NoMatch2  --> list of HSC/IR labels no mached
## labelsHSC0,labelsHSC1,labelsHSC2 --> HSC images labeled
## labelsIR0,labelsIR1,labelsIR2 --> IR images labeled
## imgsInput --> list of correct images [True, True, False]
################################################################################################     

    def matchLabels(self, grayHSC0,grayHSC1,grayHSC2,grayIR0,grayIR1,grayIR2,offsetX,offsetY):

        imgsInput = [False] *3

        if (len(grayHSC0)>0) and (len(grayIR0)>0):
            imgsInput[0]=True
            sizeHSC = grayHSC0.shape
            sizeIR = grayIR0.shape
        if (len(grayHSC1)>0) and (len(grayIR1)>0):
            imgsInput[1]=True
            sizeHSC = grayHSC1.shape
            sizeIR = grayIR1.shape
        if (len(grayHSC2)>0) and (len(grayIR2)>0):
            imgsInput[2]=True
            sizeHSC = grayHSC2.shape
            sizeIR = grayIR2.shape

        h = sizeHSC[0]
        grayHSC_black = np.zeros(sizeHSC).astype(np.uint16)
        grayIR_black = np.zeros(sizeIR).astype(np.uint16)
        

        if (imgsInput[0] and imgsInput[1] and imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC0, grayHSC1, grayHSC2])        
            grayIR = cv2.vconcat([grayIR0, grayIR1, grayIR2])
        elif (not imgsInput[0] and imgsInput[1] and imgsInput[2]): 
            grayHSC = cv2.vconcat([grayHSC_black, grayHSC1, grayHSC2])          
            grayIR = cv2.vconcat([grayIR_black, grayIR1, grayIR2])
        elif (imgsInput[0] and not imgsInput[1] and imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC0, grayHSC_black, grayHSC2])
            grayIR = cv2.vconcat([grayIR0, grayIR_black, grayIR2])
        elif (imgsInput[0] and imgsInput[1] and not imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC0, grayHSC1, grayHSC_black])          
            grayIR = cv2.vconcat([grayIR0, grayIR1, grayIR_black])    
        elif (imgsInput[0] and not imgsInput[1] and not imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC0, grayHSC_black, grayHSC_black])
            grayIR = cv2.vconcat([grayIR0, grayIR_black, grayIR_black])
        elif (not imgsInput[0] and imgsInput[1] and not imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC_black, grayHSC0, grayHSC_black])
            grayIR = cv2.vconcat([grayIR_black, grayIR0, grayIR_black])
        elif (not imgsInput[0] and not imgsInput[1] and imgsInput[2]):
            grayHSC = cv2.vconcat([grayHSC_black, grayHSC_black, grayHSC0])
            grayIR = cv2.vconcat([grayIR_black, grayIR_black, grayIR0])

        num_labelsHSC, labelsHSC = cv2.connectedComponents(np.uint8(grayHSC))
        num_labelsIR,  labelsIR  = cv2.connectedComponents(np.uint8(grayIR))

        labelsHSC = np.uint8(labelsHSC)
        labelsIR = np.uint8(labelsIR)
        
        #Equation of the line  y = ax + b
        MaxFactor = 2.5
        MinFactor = 1
        splitMaxSize = 3
        S = labelsHSC.shape[0]*labelsHSC.shape[1]/3
        b = MaxFactor
        a = (MinFactor-b)/S

        Match=[]
        NoMatch1 = []
        NoMatch2 = []
        tolDist = 5

        blobsImgLab1= []
        for idx in range(1,labelsHSC.max()+1):
            mass_y, mass_x = np.where(labelsHSC == idx)
            cent_x = np.average(mass_x)+offsetX
            cent_y = np.average(mass_y)+offsetY
            blobsImgLab1.append([cent_x,cent_y,len(labelsHSC[labelsHSC==idx])])


        blobsImgLab2= []
        for idx in range(1,labelsIR.max()+1):
            mass_y, mass_x = np.where(labelsIR == idx)
            cent_x = np.average(mass_x)
            cent_y = np.average(mass_y)
            blobsImgLab2.append([cent_x,cent_y,len(labelsIR[labelsIR==idx])])

        for idx1 in range(len(blobsImgLab1)):
            distMin=9999
            idx1_lab=-1
            idx2_lab=-1
            for idx2 in range(len(blobsImgLab2)):
                distX =  abs(blobsImgLab1[idx1][0]-blobsImgLab2[idx2][0])
                distY =  abs(blobsImgLab1[idx1][1]-blobsImgLab2[idx2][1])
                dist= math.sqrt(distX**2+distY**2)
                difVol = max(blobsImgLab1[idx1][2],blobsImgLab2[idx2][2])/min(blobsImgLab1[idx1][2],blobsImgLab2[idx2][2])
                sizeTolerance = blobsImgLab1[idx1][2]*a+b
                if (sizeTolerance<1):
                    sizeTolerance = 1
                if(dist<distMin  and difVol<sizeTolerance):
                    distMin = dist
                    idx1_lab = idx1
                    idx2_lab = idx2

            if(distMin<tolDist):
                Match.append([idx1_lab+1,idx2_lab+1])
            else:
                NoMatch1.append(idx1+1)

        for idx2 in range(len(blobsImgLab2)):
            finded = False
            for idx in range(len(Match)):
                if idx2+1 == Match[idx][1]:
                    finded = True
                    break
            if (not finded):
                NoMatch2.append(idx2+1)

        return Match, NoMatch1,NoMatch2,labelsHSC[:h,:],labelsHSC[h:(h*2),:],labelsHSC[(h*2):,:],labelsIR[:h,:],labelsIR[h:(h*2),:],labelsIR[(h*2):,:],imgsInput
        
        
    ##################################################################################################
    ###  Reset buttons of Charts 
    ##################################################################################################           
        
    def resetGraph(self):
        auxVal = self.saveSeriesBlobs
        self.saveSeriesBlobs = False
        self.idxColor = 0
        self.idxChartPixel = 0
        self.idxChartBlob = 0
        self.matplotlibwidget_HSC.axis.clear()
        self.matplotlibwidget_HSC.canvas.draw()              
        self.pb_resetGraph.setEnabled(False)

        self.matplotlibwidget_IR.axis.clear()
        self.matplotlibwidget_IR.canvas.draw()     
        self._ShowImages()
        self.saveSeriesBlobs = auxVal
        
        
    def _genericGetPosMouseHSC(self, event, camera ):
  
        widthLabel = self.labHSC.width()
        heightLabel =self.labHSC.height()
        widthImageHSC = self.imHyperHSC.getWidth()
        heightImageHSC = self.imHyperHSC.getHeight()
        widthImageIR = self.imHyperIR.getWidth()
        heightImageIR = self.imHyperIR.getHeight()
        
        x_HSC = int(event.pos().x()* (widthImageHSC/widthLabel))
        y_HSC = int(event.pos().y()* (heightImageHSC/heightLabel))
        x_IR = x_HSC + self._OffX_HSCvsIR
        y_IR = y_HSC + self._OffY_HSCvsIR
        posX_IR = int((widthLabel*x_IR)/widthImageIR)
        posY_IR = int((heightLabel*y_IR)/heightImageIR) 
        
        if (self.rb_graph_SelectPixel.isChecked()):
            if (self.idxChartPixel<self.MaxColor) and (self.idxColor<self.MaxColor):
                self.inspectPixels[self.idxChartPixel][0] = event.pos().x()   # Coordinate X HSC image GUI
                self.inspectPixels[self.idxChartPixel][1] = event.pos().y()   # Coordinate Y HSC image GUI 
                self.inspectPixels[self.idxChartPixel][2] = self.colors[self.idxColor]
                self.inspectPixels[self.idxChartPixel][3] = x_HSC   # Coordinate X HSC image
                self.inspectPixels[self.idxChartPixel][4] = y_HSC   # Coordinate Y HSC image
                self.inspectPixels[self.idxChartPixel][5] = posX_IR # Coordinate X IR image GUI
                self.inspectPixels[self.idxChartPixel][6] = posY_IR # Coordinate Y IR image GUI
                self.inspectPixels[self.idxChartPixel][7] = x_IR    # Coordinate X IR image
                self.inspectPixels[self.idxChartPixel][8] = y_IR    # Coordinate Y IR image
                self.inspectPixels[self.idxChartPixel][9] = camera  # 0: PreImage; 1: image; 2: PostImage

                self.idxColor += 1 
                self.idxChartPixel += 1

        elif self.checkedBackground:

            if (camera==0):
                labelHSC = self.imgLabelHSC_pre[y_HSC,x_HSC]
            elif (camera==1):
                labelHSC = self.imgLabelHSC[y_HSC,x_HSC]
            else :
                labelHSC = self.imgLabelHSC_post[y_HSC,x_HSC]
            
            if (labelHSC>0):
                labelIR = -1
                for match in self.labelsHSCIR_Match:
                    if (labelHSC == match[0]):
                        labelIR = match[1]

                # Comprova si aquest blob s'ha inspeccinat anteriorment.
                finedLabel = False
                for idx in range(self.idxChartBlob):
                    if self.inspectBlobs[idx][0] == labelHSC:
                        finedLabel = True

                if not(finedLabel):
                    self.inspectBlobs[self.idxChartBlob][0]=labelHSC
                    self.inspectBlobs[self.idxChartBlob][1]=labelIR
                    self.inspectBlobs[self.idxChartBlob][2]=self.colors[self.idxColor]

                    y_mean = []
                    y_lower_00 = []
                    y_lower_11 = []
                    y_lower_22 = []
                    y_lower_33 = []
                    y_lower_44 = []
                    y_upper_55 = []
                    y_upper_66 = []
                    y_upper_77 = []
                    y_upper_88 = []
                    y_upper_99 = []
                    for idxBand in range(self.imHyperHSC.getNumberBands()):
                        array3Imgs = np.array([])
                        for idxImg in range(3):
                            if idxImg==0 and self.imHyperHSC_Pre != -1:
                                cvImg = self.imHyperHSC_Pre.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelHSC_pre == labelHSC 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            elif idxImg==1 and self.imHyperHSC != -1:   
                                cvImg = self.imHyperHSC.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelHSC == labelHSC 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            elif idxImg==2 and self.imHyperHSC_Post != -1:   
                                cvImg = self.imHyperHSC_Post.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelHSC_post == labelHSC 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                        val = np.percentile(array3Imgs,[0,11.11,22.22, 33.33, 44.44,55.55,66.66,77.77,88.88,100 ])
                        y_mean.append(int(np.mean(array3Imgs)))
                        y_lower_00.append(int(val[0]))
                        y_lower_11.append(int(val[1]))
                        y_lower_22.append(int(val[2]))
                        y_lower_33.append(int(val[3]))
                        y_lower_44.append(int(val[4]))
                        y_upper_55.append(int(val[5]))
                        y_upper_66.append(int(val[6]))
                        y_upper_77.append(int(val[7]))
                        y_upper_88.append(int(val[8]))
                        y_upper_99.append(int(val[9]))

                    self.spectrum_HSC_mean[self.idxChartBlob] = y_mean
                    self.spectrum_HSC_lower_00[self.idxChartBlob] = y_lower_00
                    self.spectrum_HSC_lower_11[self.idxChartBlob] = y_lower_11
                    self.spectrum_HSC_lower_22[self.idxChartBlob] = y_lower_22
                    self.spectrum_HSC_lower_33[self.idxChartBlob] = y_lower_33
                    self.spectrum_HSC_lower_44[self.idxChartBlob] = y_lower_44
                    self.spectrum_HSC_upper_55[self.idxChartBlob] = y_upper_55
                    self.spectrum_HSC_upper_66[self.idxChartBlob] = y_upper_66
                    self.spectrum_HSC_upper_77[self.idxChartBlob] = y_upper_77
                    self.spectrum_HSC_upper_88[self.idxChartBlob] = y_upper_88
                    self.spectrum_HSC_upper_99[self.idxChartBlob] = y_upper_99

                    if (labelIR != -1):
                        y_mean = []
                        y_lower_00 = []
                        y_lower_11 = []
                        y_lower_22 = []
                        y_lower_33 = []
                        y_lower_44 = []
                        y_upper_55 = []
                        y_upper_66 = []
                        y_upper_77 = []
                        y_upper_88 = []
                        y_upper_99 = []
                        for idxBand in range(self.imHyperIR.getNumberBands()):
                            array3Imgs = np.array([])
                            for idxImg in range(3):
                                if idxImg==0 and self.imHyperIR_Pre != -1:
                                    cvImg = self.imHyperIR_Pre.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelIR_pre == labelIR 
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                                elif idxImg==1 and self.imHyperIR != -1:   
                                    cvImg = self.imHyperIR.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelIR == labelIR
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                                elif idxImg==2 and self.imHyperIR_Post != -1:   
                                    cvImg = self.imHyperIR_Post.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelIR_post == labelIR 
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            val = np.percentile(array3Imgs,[0,11.11,22.22, 33.33, 44.44,55.55,66.66,77.77,88.88,100 ])
                            y_mean.append(int(np.mean(array3Imgs)))
                            y_lower_00.append(int(val[0]))
                            y_lower_11.append(int(val[1]))
                            y_lower_22.append(int(val[2]))
                            y_lower_33.append(int(val[3]))
                            y_lower_44.append(int(val[4]))
                            y_upper_55.append(int(val[5]))
                            y_upper_66.append(int(val[6]))
                            y_upper_77.append(int(val[7]))
                            y_upper_88.append(int(val[8]))
                            y_upper_99.append(int(val[9]))
                            
                        self.spectrum_IR_mean[self.idxChartBlob] = y_mean
                        self.spectrum_IR_lower_00[self.idxChartBlob] = y_lower_00
                        self.spectrum_IR_lower_11[self.idxChartBlob] = y_lower_11
                        self.spectrum_IR_lower_22[self.idxChartBlob] = y_lower_22
                        self.spectrum_IR_lower_33[self.idxChartBlob] = y_lower_33
                        self.spectrum_IR_lower_44[self.idxChartBlob] = y_lower_44
                        self.spectrum_IR_upper_55[self.idxChartBlob] = y_upper_55
                        self.spectrum_IR_upper_66[self.idxChartBlob] = y_upper_66
                        self.spectrum_IR_upper_77[self.idxChartBlob] = y_upper_77
                        self.spectrum_IR_upper_88[self.idxChartBlob] = y_upper_88
                        self.spectrum_IR_upper_99[self.idxChartBlob] = y_upper_99                  

                    self.idxChartBlob += 1 
                    self.idxColor += 1  
                
        self.pb_resetGraph.setEnabled(True)  
        
        
    def _genericGetPosMouseIR(self, event, camera ):
        
        widthLabel = self.labIR.width()
        heightLabel =self.labIR.height()
        widthImageHSC = self.imHyperHSC.getWidth()
        heightImageHSC = self.imHyperHSC.getHeight()
        widthImageIR = self.imHyperIR.getWidth()
        heightImageIR = self.imHyperIR.getHeight()
        
        x_IR = int(event.pos().x()* (widthImageIR/widthLabel))
        y_IR = int(event.pos().y()* (heightImageIR/heightLabel))
        x_HSC = x_IR - self._OffX_HSCvsIR
        y_HSC = y_IR - self._OffY_HSCvsIR
        posX_HSC = int((widthLabel*x_HSC)/widthImageHSC)
        posY_HSC = int((heightLabel*y_HSC)/heightImageHSC)

        if (self.rb_graph_SelectPixel.isChecked()):
            if (self.idxChartPixel<self.MaxColor) and (self.idxColor<self.MaxColor):
                self.inspectPixels[self.idxChartPixel][0] = posX_HSC   # Coordinate X HSC image GUI
                self.inspectPixels[self.idxChartPixel][1] = posY_HSC   # Coordinate Y HSC image GUI 
                self.inspectPixels[self.idxChartPixel][2] = self.colors[self.idxColor]
                self.inspectPixels[self.idxChartPixel][3] = x_HSC   # Coordinate X HSC image
                self.inspectPixels[self.idxChartPixel][4] = y_HSC   # Coordinate Y HSC image
                self.inspectPixels[self.idxChartPixel][5] = event.pos().x() # Coordinate X IR image GUI
                self.inspectPixels[self.idxChartPixel][6] = event.pos().y() # Coordinate Y IR image GUI
                self.inspectPixels[self.idxChartPixel][7] = x_IR    # Coordinate X IR image
                self.inspectPixels[self.idxChartPixel][8] = y_IR    # Coordinate Y IR image
                self.inspectPixels[self.idxChartPixel][9] = camera  # 0: PreImage; 1: image; 2: PostImage
                        
                self.idxColor += 1 
                self.idxChartPixel += 1 
                
        elif self.checkedBackground:

            if (camera==0):
                labelIR = self.imgLabelIR_pre[y_IR,x_IR]
            elif (camera==1):
                labelIR = self.imgLabelIR[y_IR,x_IR]
            else :
                labelIR = self.imgLabelIR_post[y_IR,x_IR]
            
            if (labelIR>0):
                labelHSC = -1
                for match in self.labelsHSCIR_Match:
                    if (labelIR == match[1]):
                        labelHSC = match[0]

                # Comprova si aquest blob s'ha inspeccinat anteriorment.
                finedLabel = False
                for idx in range(self.idxChartBlob):
                    if self.inspectBlobs[idx][1] == labelIR:
                        finedLabel = True

                if not(finedLabel):
                    self.inspectBlobs[self.idxChartBlob][0]=labelHSC
                    self.inspectBlobs[self.idxChartBlob][1]=labelIR
                    self.inspectBlobs[self.idxChartBlob][2]=self.colors[self.idxColor]

                    y_mean = []
                    y_lower_00 = []
                    y_lower_11 = []
                    y_lower_22 = []
                    y_lower_33 = []
                    y_lower_44 = []
                    y_upper_55 = []
                    y_upper_66 = []
                    y_upper_77 = []
                    y_upper_88 = []
                    y_upper_99 = []
                    
                    for idxBand in range(self.imHyperIR.getNumberBands()):
                        array3Imgs = np.array([])
                        for idxImg in range(3):
                            if idxImg==0 and self.imHyperIR_Pre != -1:
                                cvImg = self.imHyperIR_Pre.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelIR_pre == labelIR 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            elif idxImg==1 and self.imHyperIR != -1:   
                                cvImg = self.imHyperIR.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelIR == labelIR 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            elif idxImg==2 and self.imHyperIR_Post != -1:   
                                cvImg = self.imHyperIR_Post.getBWimage(idx=idxBand)
                                idxBlob = self.imgLabelIR_post == labelIR 
                                array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                        val = np.percentile(array3Imgs,[0,11.11,22.22, 33.33, 44.44,55.55,66.66,77.77,88.88,100 ])
                        y_mean.append(int(np.mean(array3Imgs)))
                        y_lower_00.append(int(val[0]))
                        y_lower_11.append(int(val[1]))
                        y_lower_22.append(int(val[2]))
                        y_lower_33.append(int(val[3]))
                        y_lower_44.append(int(val[4]))
                        y_upper_55.append(int(val[5]))
                        y_upper_66.append(int(val[6]))
                        y_upper_77.append(int(val[7]))
                        y_upper_88.append(int(val[8]))
                        y_upper_99.append(int(val[9]))
                    
                    self.spectrum_IR_mean[self.idxChartBlob] = y_mean
                    self.spectrum_IR_lower_00[self.idxChartBlob] = y_lower_00
                    self.spectrum_IR_lower_11[self.idxChartBlob] = y_lower_11
                    self.spectrum_IR_lower_22[self.idxChartBlob] = y_lower_22
                    self.spectrum_IR_lower_33[self.idxChartBlob] = y_lower_33
                    self.spectrum_IR_lower_44[self.idxChartBlob] = y_lower_44
                    self.spectrum_IR_upper_55[self.idxChartBlob] = y_upper_55
                    self.spectrum_IR_upper_66[self.idxChartBlob] = y_upper_66
                    self.spectrum_IR_upper_77[self.idxChartBlob] = y_upper_77
                    self.spectrum_IR_upper_88[self.idxChartBlob] = y_upper_88
                    self.spectrum_IR_upper_99[self.idxChartBlob] = y_upper_99
                    
                    if (labelHSC != -1):
                        y_mean = []
                        y_lower_00 = []
                        y_lower_11 = []
                        y_lower_22 = []
                        y_lower_33 = []
                        y_lower_44 = []
                        y_upper_55 = []
                        y_upper_66 = []
                        y_upper_77 = []
                        y_upper_88 = []
                        y_upper_99 = []
                        for idxBand in range(self.imHyperHSC.getNumberBands()):
                            array3Imgs = np.array([])
                            for idxImg in range(3):
                                if idxImg==0 and self.imHyperHSC_Pre != -1:
                                    cvImg = self.imHyperHSC_Pre.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelHSC_pre == labelHSC 
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                                elif idxImg==1 and self.imHyperHSC != -1:   
                                    cvImg = self.imHyperHSC.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelHSC == labelHSC
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                                elif idxImg==2 and self.imHyperHSC_Post != -1:   
                                    cvImg = self.imHyperHSC_Post.getBWimage(idx=idxBand)
                                    idxBlob = self.imgLabelHSC_post == labelHSC 
                                    array3Imgs = np.concatenate((array3Imgs, cvImg[idxBlob]), axis=0)
                            val = np.percentile(array3Imgs,[0,11.11,22.22, 33.33, 44.44,55.55,66.66,77.77,88.88,100 ])
                            y_mean.append(int(np.mean(array3Imgs)))
                            y_lower_00.append(int(val[0]))
                            y_lower_11.append(int(val[1]))
                            y_lower_22.append(int(val[2]))
                            y_lower_33.append(int(val[3]))
                            y_lower_44.append(int(val[4]))
                            y_upper_55.append(int(val[5]))
                            y_upper_66.append(int(val[6]))
                            y_upper_77.append(int(val[7]))
                            y_upper_88.append(int(val[8]))
                            y_upper_99.append(int(val[9]))
 
                        self.spectrum_HSC_mean[self.idxChartBlob] = y_mean
                        self.spectrum_HSC_lower_00[self.idxChartBlob] = y_lower_00
                        self.spectrum_HSC_lower_11[self.idxChartBlob] = y_lower_11
                        self.spectrum_HSC_lower_22[self.idxChartBlob] = y_lower_22
                        self.spectrum_HSC_lower_33[self.idxChartBlob] = y_lower_33
                        self.spectrum_HSC_lower_44[self.idxChartBlob] = y_lower_44
                        self.spectrum_HSC_upper_55[self.idxChartBlob] = y_upper_55
                        self.spectrum_HSC_upper_66[self.idxChartBlob] = y_upper_66
                        self.spectrum_HSC_upper_77[self.idxChartBlob] = y_upper_77
                        self.spectrum_HSC_upper_88[self.idxChartBlob] = y_upper_88
                        self.spectrum_HSC_upper_99[self.idxChartBlob] = y_upper_99                  
                        
                    self.idxChartBlob += 1 
                    self.idxColor += 1  
                    
            self.pb_resetGraph.setEnabled(True)
            
    ##################################################################################################
    ###  Capture mouse click on image to show pixel banding. 
    ###  The theoretical coordinate must be calculated
    ##################################################################################################           
        
    def getPosMouseHSC(self, event):
        self._genericGetPosMouseHSC(event, 1)
        self._ShowImages()
        
        
    def getPosMouseHSC_pre(self, event):
        self._genericGetPosMouseHSC(event, 0)
        self._ShowImages()
        
        
    def getPosMouseHSC_post(self, event):
        self._genericGetPosMouseHSC(event, 2)
        self._ShowImages()
        
        
    def getPosMouseIR(self, event):
        self._genericGetPosMouseIR(event, 1)
        self._ShowImages()        
        
        
    def getPosMouseIR_pre(self, event):
        self._genericGetPosMouseIR(event, 0)
        self._ShowImages()   

        
    def getPosMouseIR_post(self, event):
        self._genericGetPosMouseIR(event, 2)
        self._ShowImages()   
                             

    ##################################################################################################
    ###  Refreshes the image showing the image corresponding to the selected band  (Horizontal Slider)
    ##################################################################################################    
    
    def getHorSlider_HSC_band1(self):
#        print('Call: getHorSlider_HSC_band1()')   
        
        if (self.horSlider_HSC_band1.isEnabled()):
            widthImageHSC = self.imHyperHSC.getWidth()
            heightImageHSC = self.imHyperHSC.getHeight()

            index = self.horSlider_HSC_band1.value()-1
            self.lineEdit_HSC_idx1.setText(str(index+1))
            self.lineEdit_HSC_band1.setText(str(self.bandsF10[index])+' nm')

            self._ShowImages()

                            
    def getHorSlider_IR_band1(self):
#        print('Call: getHorSlider_IR_band1()')        

        if (self.horSlider_IR_band1.isEnabled()):
            widthImageIR = self.imHyperIR.getWidth()
            heightImageIR = self.imHyperIR.getHeight()
            
            index = self.horSlider_IR_band1.value()-1
            self.lineEdit_IR_idx1.setText(str(index+1))
            self.lineEdit_IR_band1.setText(str(self.bandsF17[index])+' nm')

            self._ShowImages()

    
    ##################################################################################################
    ###  Funtions to read configuration files of FX10 and FX17 cameras 
    ##################################################################################################
        
    #== read_F10_File ================================================================================
    #== Read the F10 camera file to identify which bands it is scanning.
    #== Configures the parameters of the 'Horizontal Slider' that selects the band to display.
    #=================================================================================================
    def read_F10_File(self):
#        print('Call: read_F10_File()')        
        
        file_dialog = QFileDialog(self)
        # the name filters must be a list
        file_dialog.setNameFilters(["All files (*.*)", "Images (*.wls)"])
        file_dialog.selectNameFilter("Images (*.wls)")
        # show the dialog
        selected = file_dialog.exec()
        if selected:
            filename = file_dialog.selectedFiles()[0]
            self.lab_FX10_pathFile.setText(filename)
            self.F10FilePath = filename
            self.pButt_HSCFile.setEnabled(True)
            self.label_HSC.setEnabled(True)
            self.lab_HSC_pathFile.setEnabled(True)
            self.F10_readed=True

            # Only if you have read the two configuration files (F10 and F17) is it possible to inspect the images in a folder
            if self.F10_readed and self.F17_readed:
                self.lab_RequiresMessage.hide()
                self.pButt_SelectFolder.setEnabled(True)
            
            # Read bands from the Camera bands file.
            self.bandsF10 = np.array([])
            with open(filename, "r") as file1:
                idx = 0
                for line in file1.readlines():
                    values = (str.split(line))                    
                    wavelength = float(values[0])
                    self.bandsF10 = np.append(self.bandsF10, wavelength) 
                    
            # Update configuration of 'Horizontal Slider' HSC_band1
            self.horSlider_HSC_band1.setMinimum(1)
            self.horSlider_HSC_band1.setMaximum(len(self.bandsF10))
            self.horSlider_HSC_band1.setValue(int(len(self.bandsF10)/2)) 
            self.lineEdit_HSC_idx1.setText(str(self.horSlider_HSC_band1.value()))
            self.lineEdit_HSC_band1.setText(str(self.bandsF10[int(len(self.bandsF10)/2)])+' nm')
            
    #== read_F17_File ================================================================================
    #== Read the F17 camera file to identify which bands it is scanning.
    #== Configures the parameters of the 'Horizontal Slider' that selects the band to display.
    #=================================================================================================
    def read_F17_File(self):
#        print('Call: read_F17_File()')        

        file_dialog = QFileDialog(self)
        # the name filters must be a list
        file_dialog.setNameFilters(["All files (*.*)", "Images (*.wls)"])
        file_dialog.selectNameFilter("Images (*.wls)")
        # show the dialog
        selected = file_dialog.exec()
        if selected:
            filename = file_dialog.selectedFiles()[0]
            self.lab_FX17_pathFile.setText(filename)
            self.F17FilePath = filename
            self.pButt_IRFile.setEnabled(True)
            self.label_IR.setEnabled(True)
            self.lab_IR_pathFile.setEnabled(True)
            self.F17_readed=True

            if self.F10_readed and self.F17_readed:
                self.lab_RequiresMessage.hide()
                self.pButt_SelectFolder.setEnabled(True)
            
            # Read bands from the Camera bands file.
            self.bandsF17 = np.array([])
            with open(filename, "r") as file1:
                for line in file1.readlines():
                    values = (str.split(line))
                    self.bandsF17 = np.append(self.bandsF17, float(values[0]))    
                    
            # Update configuration of 'Horizontal Slider' IR_band1
            self.horSlider_IR_band1.setMinimum(1)
            self.horSlider_IR_band1.setMaximum(len(self.bandsF17))
            self.horSlider_IR_band1.setValue(int(len(self.bandsF17)/2))            
            self.lineEdit_IR_idx1.setText(str(self.horSlider_IR_band1.value()))
            self.lineEdit_IR_band1.setText(str(self.bandsF17[int(len(self.bandsF17)/2)])+' nm')
        
        
    ##################################################################################################
    ###  Funtions to read images color, HSC and IR
    ##################################################################################################
    
    def read_Color_Image(self):
#       print('Call: read_Color_Image()')        

        file_dialog = QFileDialog(self)
        # the name filters must be a list
        file_dialog.setNameFilters(["All files (*.*)", "Images (*.tiff)"])
        file_dialog.selectNameFilter("Images (*.tiff)")
        # show the dialog
        selected = file_dialog.exec()
        if selected:
            filename = file_dialog.selectedFiles()[0]
            self.lab_Color_pathFile.setText(filename)
            self.color_ImagePath = filename
            self.labColor.setPixmap(QtGui.QPixmap(self.color_ImagePath))
            img = cv2.imread(self.color_ImagePath)
            h, w, c = img.shape
            self.lab_Color_width.setEnabled(True)
            self.lab_Color_width.setText(str(w))
            self.lab_Color_Height.setEnabled(True)
            self.lab_Color_Height.setText(str(h))
            self.lab_Color_Bands.setEnabled(True)
            self.lab_Color_Bands.setText(str(c))
            self.label_1_color.setEnabled(True)
            self.label_2_color.setEnabled(True)
            self.label_3_color.setEnabled(True)   
            self.labColor.setEnabled(True)
    
            self.labColor_pre.clear()
            self.labColor_pre.setEnabled(False)
            self.labColor_pre.setText('Pre-Image')
            self.labColor_post.clear()
            self.labColor_post.setEnabled(False)
            self.labColor_post.setText('Pre-Image')
            
    
    def read_HSC_Image(self):
#        print('Call: read_HSC_Image()')        

        file_dialog = QFileDialog(self)
        # the name filters must be a list
        file_dialog.setNameFilters(["All files (*.*)", "Images (*.tiff)"])
        file_dialog.selectNameFilter("Images (*.tiff)")
        # show the dialog
        selected = file_dialog.exec()
        if selected:
            filename = file_dialog.selectedFiles()[0]
            self.lab_HSC_pathFile.setText(filename)
            self.HSC_imagePath = filename
            self.imHyperHSC = HypSpImage(self.HSC_imagePath,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            self.imHyperHSC_enabled=True
            cvImg = self.imHyperHSC.getBWimage(idx=100)
            cvImg= (cvImg/np.max(cvImg)*255.0).astype(np.uint8)
            self.imgScaledHSC = cv2.resize(cvImg, (self.labHSC.width()-2, self.labHSC.height()-2), interpolation = cv2.INTER_AREA)  #JRM
            qImg = QtGui.QImage(self.imgScaledHSC, self.imgScaledHSC.shape[1], self.imgScaledHSC.shape[0], QImage.Format_Grayscale8)
            pixmap = QPixmap.fromImage(qImg)
            self.labHSC.setPixmap(pixmap)
            self.horSlider_HSC_band1.setEnabled(True)
            self.horSlider_HSC_band1.setMinimum(1)
            self.horSlider_HSC_band1.setMaximum(len(self.bandsF10))
            self.horSlider_HSC_band1.setValue(50)
            self.lineEdit_HSC_band1.setEnabled(True)
            self.lineEdit_HSC_band1.setText(str(self.bandsF10[0])+' nm')
            self.lineEdit_HSC_idx2.setText(str(self.horSlider_HSC_band2.value()))
            self.lineEdit_HSC_band2.setText(str(self.bandsF10[0]))
            self.lab_HSC_width.setEnabled(True)
            self.lab_HSC_width.setText(str(self.imHyperHSC.getWidth()))
            self.lab_HSC_Height.setEnabled(True)
            self.lab_HSC_Height.setText(str(self.imHyperHSC.getHeight()))
            self.lab_HSC_Bands.setEnabled(True)
            self.lab_HSC_Bands.setText(str(self.imHyperHSC.getNumberBands()))
            self.lab_HSC_MinBand.setEnabled(True)
            self.lab_HSC_MinBand.setText(str(self.bandsF10[0]))
            self.lab_HSC_MaxBand.setEnabled(True)
            self.lab_HSC_MaxBand.setText(str(self.bandsF10[-1]))
            self.labHSC.setEnabled(True)
            self.label_1_HSC.setEnabled(True)
            self.label_2_HSC.setEnabled(True)
            self.label_3_HSC.setEnabled(True)
            self.label_4_HSC.setEnabled(True)
            self.label_5_HSC.setEnabled(True)
            self.pb_resetGraph.setEnabled(False)
            self.idxChartPixel = 0
            self.matplotlibwidget_HSC.axis.clear()
            self.matplotlibwidget_HSC.canvas.draw()                         

            self.labHSC_pre.clear()
            self.labHSC_pre.setEnabled(False)
            self.labHSC_pre.setText('Pre-Image')
            self.labHSC_post.clear()
            self.labHSC_post.setEnabled(False)
            self.labHSC_post.setText('Pre-Image')
            self.imHyperHSC_Pre_enabled = False
            self.imHyperHSC_Post_enabled = False
            
            
    def read_IR_Image(self):
#        print('Call: read_IR_Image()')        
        
        file_dialog = QFileDialog(self)
        # the name filters must be a list
        file_dialog.setNameFilters(["All files (*.*)", "Images (*.tiff)"])
        file_dialog.selectNameFilter("Images (*.tiff)")
        # show the dialog
        selected = file_dialog.exec()
        if selected:
            filename = file_dialog.selectedFiles()[0]
            self.lab_IR_pathFile.setText(filename)
            self.IR_imagePath = filename
            self.imHyperIR = HypSpImage(self.IR_imagePath,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            self.imHyperIR_enabled=True
            cvImg = self.imHyperIR.getBWimage(idx=100)
            cvImg= (cvImg/np.max(cvImg)*255.0).astype(np.uint8)
            qImg = QtGui.QImage(cvImg, cvImg.shape[1], cvImg.shape[0], QImage.Format_Grayscale8)
            pixmap = QPixmap.fromImage(qImg)
            self.labIR.setPixmap(pixmap)
            self.horSlider_IR_band1.setEnabled(True)
            self.horSlider_IR_band1.setMinimum(1)
            self.horSlider_IR_band1.setMaximum(len(self.bandsF17))
            self.horSlider_IR_band1.setValue(50)
            self.lineEdit_IR_band1.setEnabled(True)
            self.lineEdit_IR_band1.setText(str(self.bandsF17[0])+' nm')
            self.lab_IR_width.setEnabled(True)
            self.lab_IR_width.setText(str(self.imHyperIR.getWidth()))
            self.lab_IR_Height.setEnabled(True)
            self.lab_IR_Height.setText(str(self.imHyperIR.getHeight()))
            self.lab_IR_Bands.setEnabled(True)
            self.lab_IR_Bands.setText(str(self.imHyperIR.getNumberBands()))
            self.lab_IR_MinBand.setEnabled(True)
            self.lab_IR_MinBand.setText(str(self.bandsF17[0]))
            self.lab_IR_MaxBand.setEnabled(True)
            self.lab_IR_MaxBand.setText(str(self.bandsF17[-1]))
            self.labIR.setEnabled(True)
            self.label_1_IR.setEnabled(True)
            self.label_2_IR.setEnabled(True)
            self.label_3_IR.setEnabled(True)
            self.label_4_IR.setEnabled(True)
            self.label_5_IR.setEnabled(True)
            self.rb_IR_band1.setEnabled(True)
            self.matplotlibwidget_IR.axis.clear()
            self.matplotlibwidget_IR.canvas.draw()                 
            
            self.labIR_pre.clear()
            self.labIR_pre.setEnabled(False)
            self.labIR_pre.setText('Pre-Image')
            self.labIR_post.clear()
            self.labIR_post.setEnabled(False)
            self.labIR_post.setText('Pre-Image')
            self.imHyperIR_Pre_enabled = False
            self.imHyperIR_Post_enabled = False

            
        
    ##################################################################################################
    ###  Funtions to read images in folder 
    ##################################################################################################
  
    #== selectFolder =================================================================================
    #== Inspect a forlder for to find images composed of 'Color', HSC and IR files 
    #=================================================================================================
    def selectFolder(self):
#        print('Call: selectFolder()')    
        
        # Opens the dialog window to select the folder that contains the images to be inspected
        self.folderpath = QFileDialog.getExistingDirectory(self, 'Select Folder')
        self.lab_pathFolder.setText(self.folderpath)
        
        # Initializes the 'Color' image information
        self.listImgs=[]
        self.totalImgs = 0
        self.lab_Color_width.setEnabled(False)
        self.lab_Color_width.setText('-----')
        self.lab_Color_Height.setEnabled(False)
        self.lab_Color_Height.setText('-----')
        self.lab_Color_Bands.setEnabled(False)
        self.lab_Color_Bands.setText('-----')   
        self.label_1_color.setEnabled(False)
        self.label_2_color.setEnabled(False)
        self.label_3_color.setEnabled(False)  
        
        # Initializes the 'HSC' image information
        self.lab_HSC_width.setEnabled(False)
        self.lab_HSC_width.setText('-----')
        self.lab_HSC_Height.setEnabled(False)
        self.lab_HSC_Height.setText('-----')
        self.lab_HSC_Bands.setEnabled(False)
        self.lab_HSC_Bands.setText('-----')
        self.lab_HSC_MinBand.setEnabled(False)
        self.lab_HSC_MinBand.setText('-----')
        self.lab_HSC_MaxBand.setEnabled(True)
        self.lab_HSC_MaxBand.setText('-----')        
        self.label_1_HSC.setEnabled(False)
        self.label_2_HSC.setEnabled(False)
        self.label_3_HSC.setEnabled(False)
        self.label_4_HSC.setEnabled(False)
        self.label_5_HSC.setEnabled(False) 
        
        # Initializes the 'IR' image information
        self.lab_IR_width.setEnabled(False)
        self.lab_IR_width.setText('-----')
        self.lab_IR_Height.setEnabled(False)
        self.lab_IR_Height.setText('-----')
        self.lab_IR_Bands.setEnabled(False)
        self.lab_IR_Bands.setText('-----')
        self.lab_IR_MinBand.setEnabled(False)
        self.lab_IR_MinBand.setText('-----')
        self.lab_IR_MaxBand.setEnabled(False)
        self.lab_IR_MaxBand.setText('-----')        
        self.label_1_IR.setEnabled(False)
        self.label_2_IR.setEnabled(False)
        self.label_3_IR.setEnabled(False)
        self.label_4_IR.setEnabled(False)
        self.label_5_IR.setEnabled(False)         
        
        # Clear the frame where the 'Color' images are displayed
        self.labColor_pre.clear()        
        self.labColor_pre.setEnabled(False)
        self.labColor_pre.setText('Pre-Image')
        self.labColor.clear()        
        self.labColor.setEnabled(False)
        self.labColor.setText('Image')
        self.labColor_post.clear()        
        self.labColor_post.setEnabled(False)
        self.labColor_post.setText('Post-Image')
        
        # Clear the frame where the 'HSC' images are displayed
        self.labHSC_pre.clear()
        self.labHSC_pre.setEnabled(False)
        self.labHSC_pre.setText('Pre-Image')
        self.labHSC.clear()
        self.labHSC.setEnabled(False)
        self.labHSC.setText('Image')
        self.labHSC_post.clear()
        self.labHSC_post.setEnabled(False)
        self.labHSC_post.setText('Post-Image')
        
        # Clear the frame where the 'IR' images are displayed
        self.labIR_pre.clear()
        self.labIR_pre.setEnabled(False)
        self.labIR_pre.setText('Pre-Image')        
        self.labIR.clear()
        self.labIR.setEnabled(False)
        self.labIR.setText('Image')        
        self.labIR_post.clear()
        self.labIR_post.setEnabled(False)
        self.labIR_post.setText('Post-Image')        
        
        # Clears the list that shows the images in the folder
        self.listWidget.clear()

        # Specifies whether to display the image or not (the selected image is displayed in the center 
        # frame and the pre-image or post-image may not exist, in which case it should not be displayed)
        self.imHyperHSC_Pre_enabled = False
        self.imHyperHSC_enabled = False
        self.imHyperHSC_Post_enabled = False
        self.imHyperIR_Pre_enabled = False
        self.imHyperIR_enabled = False
        self.imHyperIR_Post_enabled = False
        
        # InspeInspect the folder to identify the corresponding images (HSC, IR and color files are required for each image)
        for file in os.listdir(self.folderpath):
            result = re.match('color_A_[0-9]{5}.tiff', file)
            if (result):
                strId=file[8:13]
                hsc_File = os.path.join(self.folderpath,"hsc_A_"+strId+".tiff")
                ir_File = os.path.join(self.folderpath,"ir_A_"+strId+".tiff")
                
                if os.path.isfile(hsc_File) and os.path.isfile(ir_File):
                    self.listWidget.addItem("Images "+ file[8:13])
                    self.listImgs.append((file,hsc_File,ir_File))
                    self.totalImgs += 1
        
        # Shows the name of the selected image and its number                   # AIXÒ ES PODRIA ELIMINAR JA QUE
        if self.totalImgs == 1:                                                 # JA HO FA '_showSelectedImages'
            self.lab_PreImage.setText('---')                                    # -tret que ho hi hagui cap imatge-
            self.lab_Image.setText(self.listImgs[0][0])
            self.lab_PostImage.setText('---')
            self.lab_InspImages.setText('1 / 1')
            self.idxImgs = 0
            self.listWidget.setCurrentRow(self.idxImgs)
            self.listWidget.setFocus()  
            self.pButt_previousImage.setEnabled(False)
            self.pButt_nextImage.setEnabled(False)
            self.pButt_batchInspection.setEnabled(True)
            self.pButt_inspectBackground.setEnabled(False)
        elif self.totalImgs > 1:
            self.lab_PreImage.setText('---')
            self.lab_Image.setText(self.listImgs[0][0])  # Shows name of 'Color' file
            self.lab_PostImage.setText(self.listImgs[1][0])
            self.lab_InspImages.setText('1 / '+str(self.totalImgs))
            self.idxImgs = 0;
            self.listWidget.setCurrentRow(self.idxImgs)
            self.listWidget.setFocus() 
            self.pButt_previousImage.setEnabled(False)
            self.pButt_nextImage.setEnabled(True)
            self.pButt_batchInspection.setEnabled(True)
            self.pButt_inspectBackground.setEnabled(True)
        else:
            self.lab_PreImage.setText('---')
            self.lab_Image.setText('---')
            self.lab_PostImage.setText('---')
            self.lab_InspImages.setText('0 / 0')
            self.idxImgs = -1
            self.totalImgs = 0 
            self.pButt_previousImage.setEnabled(False)
            self.pButt_nextImage.setEnabled(False)
            self.pButt_batchInspection.setEnabled(False)
            self.pButt_inspectBackground.setEnabled(False)
          
        # If an image is displayed the 'horizontal slider' is enabled to indicate which band to display
        if (self.idxImgs != -1 ):
            self._readHypercubes(self.idxImgs,-2)
            self._ShowImages()

            self.horSlider_HSC_band1.setEnabled(True)
            self.lineEdit_HSC_band1.setEnabled(True)
            
            self.horSlider_IR_band1.setEnabled(True)
            self.lineEdit_IR_band1.setEnabled(True)

        else:
            self.horSlider_HSC_band1.setEnabled(False)
            self.lineEdit_HSC_band1.setEnabled(False)
            
            self.horSlider_IR_band1.setEnabled(False)
            self.lineEdit_IR_band1.setEnabled(False)

        self.resetGraph()
            
       
    #== _extractBackground ===============================================================================
    #== Inspects one HSC or IR hypertepectal image to determine which areas correspond to the background.
    #== It is required that the 'learningBackground' function has been previously called
    #== Camera: 1 --> HSC
    #== Camera: 2 --> IR
    #=====================================================================================================
    def _extractBackground(self, imgHyp, camera): 
    
        if (camera==1):
            imgHSC_SegmLayer = np.zeros_like(imgHyp.getBWimage(idx=0))
            imgHSC_ZerosLayer = np.zeros_like(imgHyp.getBWimage(idx=0))
            img_Bands = imgHyp.getNumberBands()  
            
            if (self.imgHyperHSC_BackGround_Max != []  and self.imgHyperHSC_BackGround_Min != []):
                #for idx in range(img_Bands):
                for idx in range(40,img_Bands-40): # Set a margin to avoid noise-containing bands
                    diff = self.imgHyperHSC_BackGround_Max[idx]-imgHyp.getBWimage(idx=idx) 
                    idxsNeg = (diff < 0)
                    imgHSC_SegmLayer[idxsNeg] = 255
                    diff = self.imgHyperHSC_BackGround_Min[idx]-imgHyp.getBWimage(idx=idx) 
                    idxsNeg = (diff > 0)
                    imgHSC_SegmLayer[idxsNeg] = 255
               
            # Remove small blobs (less than self._MIN_BLOB)
            num_labels, labels = cv2.connectedComponents(np.uint8(imgHSC_SegmLayer))
            for color in range(1 , num_labels):
                num = np.count_nonzero(labels == color)
                idx= labels == color
                if num < self._MIN_BLOB :
                    imgHSC_SegmLayer[idx] = 0
                else:
                    imgHSC_SegmLayer[idx] = 255
                    
            self.imgHSC_Binary = cv2.merge([imgHSC_SegmLayer,imgHSC_SegmLayer,imgHSC_SegmLayer]).astype(np.uint8)
            kernel = np.ones((5, 5), np.uint8)
            self.imgHSC_Binary = cv2.erode(self.imgHSC_Binary, kernel)
            img_SegmLayer = cv2.erode(imgHSC_SegmLayer, kernel)
            
            
        if (camera==2):
            imgIR_SegmLayer = np.zeros_like(imgHyp.getBWimage(idx=0))
            imgIR_ZerosLayer = np.zeros_like(imgHyp.getBWimage(idx=0))
            img_Bands = imgHyp.getNumberBands()  
            
            if (self.imgHyperIR_BackGround_Max != []  and self.imgHyperIR_BackGround_Min != []):
                #for idx in range(img_Bands):
                for idx in range(40,img_Bands-40):
                    diff = self.imgHyperIR_BackGround_Max[idx]-imgHyp.getBWimage(idx=idx)
                    idxsNeg = (diff < 0)
                    imgIR_SegmLayer[idxsNeg] = 255
                    diff = self.imgHyperIR_BackGround_Min[idx]-imgHyp.getBWimage(idx=idx)
                    idxsNeg = (diff > 0)
                    imgIR_SegmLayer[idxsNeg] = 255
                    
            # Remove small blobs (less than self._MIN_BLOB)
            num_labels, labels = cv2.connectedComponents(np.uint8(imgIR_SegmLayer))
            for color in range(1 , num_labels):
                num = np.count_nonzero(labels == color)
                idx= labels == color
                if num < self._MIN_BLOB :
                    imgIR_SegmLayer[idx] = 0
                else:
                    imgIR_SegmLayer[idx] = 255
                    
            self.imgIR_Binary = cv2.merge([imgIR_SegmLayer,imgIR_SegmLayer,imgIR_SegmLayer]).astype(np.uint8)
            kernel = np.ones((5, 5), np.uint8)
            self.imgIR_Binary = cv2.erode(self.imgIR_Binary, kernel)
            img_SegmLayer = cv2.erode(imgIR_SegmLayer, kernel)
        
        return img_SegmLayer
    

    def enhanceImage(self,camera, imgIn, imgLabel, typeEnhance, enhBlob, enhBG, strHSCIR, strOnlyHSC, strOnlyIR):
        
        if (typeEnhance==self._DO_NOT_SEGMENT):
            imgOut = imgIn
            
        elif (typeEnhance==self._BINARIZE):
            imgBin = np.copy(imgLabel)
            idx = imgBin >0
            imgBin[idx]=255
            imgOut = cv2.merge([imgBin, imgBin, imgBin])
            
        elif (typeEnhance==self._MARKS):
            alpha = 0.6
            rLayer, gLayer, bLayer=  cv2.split(imgIn)
            if (camera==self._camHSC):
                for match in strHSCIR:
                    idxsSegm = (imgLabel ==  match[0])
                    rLayer[idxsSegm] = 255
                    gLayer[idxsSegm] = 255
                    bLayer[idxsSegm] = 0            
                for match in strOnlyHSC:
                    idxsSegm = (imgLabel ==  match)
                    rLayer[idxsSegm] = 0
                    gLayer[idxsSegm] = 0
                    bLayer[idxsSegm] = 255                                
            elif (camera==self._camIR):
                for match in strHSCIR:
                    idxsSegm = (imgLabel ==  match[1])
                    rLayer[idxsSegm] = 255
                    gLayer[idxsSegm] = 255
                    bLayer[idxsSegm] = 0            
                for match in strOnlyIR:
                    idxsSegm = (imgLabel ==  match)
                    rLayer[idxsSegm] = 0
                    gLayer[idxsSegm] = 255
                    bLayer[idxsSegm] = 0            
            img_Segmentation = cv2.merge([rLayer, gLayer, bLayer])
            
            rLayer, gLayer, bLayer=  cv2.split(imgIn)
            idxsSegm = (imgLabel == 0)
            rLayer[idxsSegm] = 255
            gLayer[idxsSegm] = 0
            bLayer[idxsSegm] = 0            
            img_Background = cv2.merge([rLayer, gLayer, bLayer])
            imgOut = imgIn
            if (enhBlob):
                imgOut = cv2.addWeighted(imgOut,alpha,img_Segmentation,1-alpha,0)
            if (enhBG):
                imgOut = cv2.addWeighted(imgOut,alpha,img_Background,1-alpha,0)
            
        else:
            imgOut = imgIn
    
        return imgOut

    #== _ShowImages ==============================================================================
    #== Genera les imatges que han de ser mostrades per pantalla, així com les gràfiques aplica
    #== per aquest ordre:
    #==  1.- Obté la imatge inicial
    #==  2.- Genera les mascares
    #==        - Binari
    #==        - Mascara que resalta els objectes (linkats i no linkats)/ background
    #==        - Mascara per motrar els blobs mostrats a les gràfiques
    #==  3.- Genera el punts on s'analitza l'histograma dels pixels 
    #=================================================================================================
    def _ShowImages(self):
#        print("_ShowImages")
        
        #-- Show color images -----------------------------------------------------------------------
        if(len(self.img_Color_pre)>0):           
            height, width, channels = self.img_Color_pre.shape
            bytesPerLine = channels * width
            imgColorRGB = cv2.cvtColor(self.img_Color_pre, cv2.COLOR_BGR2RGB)
            qImg = QImage(imgColorRGB.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labColor_pre.setPixmap(pixmap)
            self.labColor_pre.setEnabled(True)
        else:
            self.labColor_pre.clear()
            self.labColor_pre.setEnabled(False)
            self.labColor_pre.setText('Pre-Image')
            
        if(len(self.img_Color)>0):           
            height, width, channels = self.img_Color.shape
            bytesPerLine = channels * width
            imgColorRGB = cv2.cvtColor(self.img_Color, cv2.COLOR_BGR2RGB)
            qImg = QImage(imgColorRGB.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labColor.setPixmap(pixmap)
            self.labColor.setEnabled(True)
        else:
            self.labColor.clear()
            self.labColor.setEnabled(False)
            self.labColor.setText('Image')
            
        if(len(self.img_Color_post)>0):           
            height, width, channels = self.img_Color_post.shape
            bytesPerLine = channels * width
            imgColorRGB = cv2.cvtColor(self.img_Color_post, cv2.COLOR_BGR2RGB)
            qImg = QImage(imgColorRGB.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labColor_post.setPixmap(pixmap)
            self.labColor_post.setEnabled(True)
        else:
            self.labColor_post.clear()
            self.labColor_post.setEnabled(False)
            self.labColor_post.setText('Post-Image')
            
        #-- Get HSC images -------------------------------------------------------------------------
        index = self.horSlider_HSC_band1.value()-1
        if(self.imHyperHSC_Pre != -1):
            cvImgHSC_Pre = self.imHyperHSC_Pre.getBWimage(idx=index)
            cvImgHSC_Pre= (cvImgHSC_Pre/np.max(cvImgHSC_Pre)*255.0).astype(np.uint8)
            cvImgHSC_Pre = cv2.cvtColor(cvImgHSC_Pre,cv2.COLOR_GRAY2RGB)

        if(self.imHyperHSC != -1):
            cvImgHSC = self.imHyperHSC.getBWimage(idx=index)
            cvImgHSC= (cvImgHSC/np.max(cvImgHSC)*255.0).astype(np.uint8)
            cvImgHSC = cv2.cvtColor(cvImgHSC,cv2.COLOR_GRAY2RGB)
            
        if(self.imHyperHSC_Post != -1):
            cvImgHSC_Post = self.imHyperHSC_Post.getBWimage(idx=index)
            cvImgHSC_Post= (cvImgHSC_Post/np.max(cvImgHSC_Post)*255.0).astype(np.uint8)
            cvImgHSC_Post = cv2.cvtColor(cvImgHSC_Post,cv2.COLOR_GRAY2RGB)
            

        #-- Get IR images -------------------------------------------------------------------------
        index = self.horSlider_IR_band1.value()-1
        if(self.imHyperIR_Pre != -1):
            cvImgIR_Pre = self.imHyperIR_Pre.getBWimage(idx=index)
            cvImgIR_Pre= (cvImgIR_Pre/np.max(cvImgIR_Pre)*255.0).astype(np.uint8)
            cvImgIR_Pre = cv2.cvtColor(cvImgIR_Pre,cv2.COLOR_GRAY2RGB)

        if(self.imHyperIR != -1):
            cvImgIR = self.imHyperIR.getBWimage(idx=index)
            cvImgIR = (cvImgIR/np.max(cvImgIR)*255.0).astype(np.uint8)
            cvImgIR = cv2.cvtColor(cvImgIR,cv2.COLOR_GRAY2RGB)

        if(self.imHyperIR_Post != -1):
            cvImgIR_Post = self.imHyperIR_Post.getBWimage(idx=index)
            cvImgIR_Post= (cvImgIR_Post/np.max(cvImgIR_Post)*255.0).astype(np.uint8)
            cvImgIR_Post = cv2.cvtColor(cvImgIR_Post,cv2.COLOR_GRAY2RGB)
        
        #============================================================================================
        #-- Show mascks in HSC images ---------------------------------------------------------------
        #-- Show mascks in IR images ----------------------------------------------------------------

        if (self.checkedBackground):
            

            if (self.rb_Segm_Bin.isChecked()):
                mode = self._BINARIZE
            elif (self.rb_Segm_Mark.isChecked()):
                mode = self._MARKS
            else:
                mode = self._DO_NOT_SEGMENT

            if(self.imHyperHSC_Pre != -1):
                cvImgHSC_Pre = self.enhanceImage(self._camHSC, cvImgHSC_Pre,self.imgLabelHSC_pre, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(), self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
                cvImgIR_Pre = self.enhanceImage(self._camIR, cvImgIR_Pre,self.imgLabelIR_pre, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(),self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
            if(self.imHyperHSC != -1):
                cvImgHSC = self.enhanceImage(self._camHSC, cvImgHSC,self.imgLabelHSC, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(), self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
                cvImgIR = self.enhanceImage(self._camIR, cvImgIR,self.imgLabelIR, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(),self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
            if(self.imHyperHSC_Post != -1):
                cvImgHSC_Post = self.enhanceImage(self._camHSC, cvImgHSC_Post,self.imgLabelHSC_post, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(), self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
                cvImgIR_Post = self.enhanceImage(self._camIR, cvImgIR_Post,self.imgLabelIR_post, mode, self.cb_MarkBlobs.isChecked(), self.cb_MarkBG.isChecked(),self.labelsHSCIR_Match,self.labelsHSC_Only, self.labelsIR_Only)
        
        
        #============================================================================================
        #-- Show analized pixels in HSC images ------------------------------------------------------

        if(self.imHyperHSC != -1):
            self.lab_HSC_width.setEnabled(True)
            self.lab_HSC_width.setText(str(self.imHyperHSC.getWidth()))
            self.lab_HSC_Height.setEnabled(True)
            self.lab_HSC_Height.setText(str(self.imHyperHSC.getHeight()))
            self.lab_HSC_Bands.setEnabled(True)
            self.lab_HSC_Bands.setText(str(self.imHyperHSC.getNumberBands()))
            self.lab_HSC_MinBand.setEnabled(True)
            self.lab_HSC_MinBand.setText(str(self.bandsF10[0]))
            self.lab_HSC_MaxBand.setEnabled(True)
            self.lab_HSC_MaxBand.setText(str(self.bandsF10[-1]))        
            self.label_1_HSC.setEnabled(True)
            self.label_2_HSC.setEnabled(True)
            self.label_3_HSC.setEnabled(True)
            self.label_4_HSC.setEnabled(True)
            self.label_5_HSC.setEnabled(True)            

        if(self.imHyperIR != -1):
            self.lab_IR_width.setEnabled(True)
            self.lab_IR_width.setText(str(self.imHyperIR.getWidth()))
            self.lab_IR_Height.setEnabled(True)
            self.lab_IR_Height.setText(str(self.imHyperIR.getHeight()))
            self.lab_IR_Bands.setEnabled(True)
            self.lab_IR_Bands.setText(str(self.imHyperIR.getNumberBands()))
            self.lab_IR_MinBand.setEnabled(True)
            self.lab_IR_MinBand.setText(str(self.bandsF17[0]))
            self.lab_IR_MaxBand.setEnabled(True)
            self.lab_IR_MaxBand.setText(str(self.bandsF17[-1]))        
            self.label_1_IR.setEnabled(True)
            self.label_2_IR.setEnabled(True)
            self.label_3_IR.setEnabled(True)
            self.label_4_IR.setEnabled(True)
            self.label_5_IR.setEnabled(True)            
            
        if(self.imHyperHSC_Pre != -1):
            self.imgScaledHSC_Pre = cv2.resize(cvImgHSC_Pre, (self.labHSC.width()-2, self.labHSC.height()-2), interpolation = cv2.INTER_AREA)
        if(self.imHyperHSC != -1):
            self.imgScaledHSC = cv2.resize(cvImgHSC, (self.labHSC.width()-2, self.labHSC.height()-2), interpolation = cv2.INTER_AREA)
        if(self.imHyperHSC_Post != -1):
            self.imgScaledHSC_Post = cv2.resize(cvImgHSC_Post, (self.labHSC.width()-2, self.labHSC.height()-2), interpolation = cv2.INTER_AREA)
        if(self.imHyperIR_Pre != -1):
            self.imgScaledIR_Pre = cv2.resize(cvImgIR_Pre, (self.labIR.width()-2, self.labIR.height()-2), interpolation = cv2.INTER_AREA)
        if(self.imHyperIR != -1):
            self.imgScaledIR = cv2.resize(cvImgIR, (self.labIR.width()-2, self.labIR.height()-2), interpolation = cv2.INTER_AREA)
        if(self.imHyperIR_Post != -1):
            self.imgScaledIR_Post = cv2.resize(cvImgIR_Post, (self.labIR.width()-2, self.labIR.height()-2), interpolation = cv2.INTER_AREA)
                
        thickness = 1
        for idx in range(self.idxChartPixel):
            start_point = (self.inspectPixels[idx][0]-2, self.inspectPixels[idx][1]-2)
            end_point = (self.inspectPixels[idx][0]+2, self.inspectPixels[idx][1]+2)
            col=tuple([int(c * 255) for c in colors.to_rgb(self.inspectPixels[idx][2])])            
            if(self.inspectPixels[idx][9]==0) and (self.imHyperHSC_Pre != -1):
                self.imgScaledHSC_Pre = cv2.rectangle(self.imgScaledHSC_Pre, start_point, end_point, col , thickness)
            if(self.inspectPixels[idx][9]==1) and (self.imHyperHSC != -1):
                self.imgScaledHSC = cv2.rectangle(self.imgScaledHSC, start_point, end_point, col , thickness)
            if(self.inspectPixels[idx][9]==2) and (self.imHyperHSC_Post != -1):
                self.imgScaledHSC_Post = cv2.rectangle(self.imgScaledHSC_Post, start_point, end_point, col , thickness)

            start_point = (self.inspectPixels[idx][5]-2 , self.inspectPixels[idx][6]-2)
            end_point = (self.inspectPixels[idx][5]+2 , self.inspectPixels[idx][6]+2)
            col=tuple([int(c * 255) for c in colors.to_rgb(self.inspectPixels[idx][2])])
            if(self.inspectPixels[idx][9]==0) and (self.imHyperIR_Pre != -1):
                self.imgScaledIR_Pre = cv2.rectangle(self.imgScaledIR_Pre, start_point, end_point, col , thickness)
            if(self.inspectPixels[idx][9]==1) and (self.imHyperIR != -1):
                self.imgScaledIR = cv2.rectangle(self.imgScaledIR, start_point, end_point, col , thickness)
            if(self.inspectPixels[idx][9]==2) and (self.imHyperIR_Post != -1):
                self.imgScaledIR_Post = cv2.rectangle(self.imgScaledIR_Post, start_point, end_point, col , thickness)
        
        #============================================================================================
        #-- Show HSC images -------------------------------------------------------------------------
        if(self.imHyperHSC_Pre != -1):
            height, width, channels = self.imgScaledHSC_Pre.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledHSC_Pre.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labHSC_pre.setPixmap(pixmap)
            self.labHSC_pre.setEnabled(True)
        else:
            self.labHSC_pre.clear()
            self.labHSC_pre.setEnabled(False)
            self.labHSC_pre.setText('Pre-Image')
        if(self.imHyperHSC != -1):
            height, width, channels = self.imgScaledHSC.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledHSC.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labHSC.setPixmap(pixmap)
            self.labHSC.setEnabled(True)
        else:
            self.labHSC.clear()
            self.labHSC.setEnabled(False)
            self.labHSC.setText('Image')
            
        if(self.imHyperHSC_Post != -1):
            height, width, channels = self.imgScaledHSC_Post.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledHSC_Post.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labHSC_post.setPixmap(pixmap)
            self.labHSC_post.setEnabled(True)
        else:
            self.labHSC_post.clear()
            self.labHSC_post.setEnabled(False)
            self.labHSC_post.setText('Post-Image')
            

        #-- Show IR images -------------------------------------------------------------------------
        if(self.imHyperIR_Pre != -1):
            height, width, channels = self.imgScaledIR_Pre.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledIR_Pre.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labIR_pre.setPixmap(pixmap)
            self.labIR_pre.setEnabled(True)
        else:
            self.labIR_pre.clear()
            self.labIR_pre.setEnabled(False)
            self.labIR_pre.setText('Pre-Image')            

        if(self.imHyperIR != -1):
            height, width, channels = self.imgScaledIR.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledIR.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labIR.setPixmap(pixmap)
            self.labIR.setEnabled(True)
        else:
            self.labIR.clear()
            self.labIR.setEnabled(False)
            self.labIR.setText('Image')            

        if(self.imHyperIR_Post != -1):
            height, width, channels = self.imgScaledIR_Post.shape
            bytesPerLine = channels * width
            qImg = QImage(self.imgScaledIR_Post.data, width, height, bytesPerLine, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qImg)
            self.labIR_post.setPixmap(pixmap)
            self.labIR_post.setEnabled(True)
        else:
            self.labIR_post.clear()
            self.labIR_post.setEnabled(False)
            self.labIR_post.setText('Post-Image')     

        #-- Show HSC and IR Graphs -------------------------------------------------------------------------
        # JRM
        widthImageHSC = self.imHyperHSC.getWidth()
        heightImageHSC = self.imHyperHSC.getHeight()
        widthImageIR = self.imHyperIR.getWidth()
        heightImageIR = self.imHyperIR.getHeight()
        
        self.matplotlibwidget_HSC.axis.clear()
        drawGraph = False
        for idx in range(self.idxChartPixel):            
            if ((self.inspectPixels[idx][3]>=0) and (self.inspectPixels[idx][3]<widthImageHSC) and (self.inspectPixels[idx][4]>=0) and (self.inspectPixels[idx][4]<heightImageHSC)):
                x_values = np.linspace(self.bandsF10[0], self.bandsF10[-1], self.bandsF10.shape[0])
                if (self.inspectPixels[idx][9]==0):
                    y_values = self.imHyperHSC_Pre.getPixelBands(self.inspectPixels[idx][3]-1,self.inspectPixels[idx][4]-1)
                if (self.inspectPixels[idx][9]==1):
                    y_values = self.imHyperHSC.getPixelBands(self.inspectPixels[idx][3]-1,self.inspectPixels[idx][4]-1)
                if (self.inspectPixels[idx][9]==2):
                    y_values = self.imHyperHSC_Post.getPixelBands(self.inspectPixels[idx][3]-1,self.inspectPixels[idx][4]-1)
                self.matplotlibwidget_HSC.axis.plot(x_values,y_values,color=self.inspectPixels[idx][2])
                drawGraph = True
        
        for idxColor in range(self.idxChartBlob):
            if(self.inspectBlobs[idxColor][0] >0):
                x_values = np.linspace(self.bandsF10[0], self.bandsF10[-1], self.bandsF10.shape[0])
                self.matplotlibwidget_HSC.axis.fill_between(x_values, self.spectrum_HSC_lower_00[idxColor], self.spectrum_HSC_upper_99[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_HSC.axis.fill_between(x_values, self.spectrum_HSC_lower_11[idxColor], self.spectrum_HSC_upper_88[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_HSC.axis.fill_between(x_values, self.spectrum_HSC_lower_22[idxColor], self.spectrum_HSC_upper_77[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_HSC.axis.fill_between(x_values, self.spectrum_HSC_lower_33[idxColor], self.spectrum_HSC_upper_66[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_HSC.axis.fill_between(x_values, self.spectrum_HSC_lower_44[idxColor], self.spectrum_HSC_upper_55[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_HSC.axis.plot(x_values, self.spectrum_HSC_mean[idxColor],color=self.inspectBlobs[idxColor][2])                        
                drawGraph = True

        if (drawGraph):
            self.matplotlibwidget_HSC.axis.plot(x_values,[0]*self.imHyperHSC.getNumberBands(), linewidth=1,color='k')# JRM
            self.matplotlibwidget_HSC.axis.axvline(x=self.bandsF10[self.horSlider_HSC_band1.value()-1])
            self.matplotlibwidget_HSC.axis.axvline(x=self._WAVELENGTH_RED,color='r')
            self.matplotlibwidget_HSC.axis.axvline(x=self._WAVELENGTH_GREEN,color='g')
            self.matplotlibwidget_HSC.axis.axvline(x=self._WAVELENGTH_BLUE,color='b')
            self.matplotlibwidget_HSC.canvas.draw() 
            self.pb_resetGraph.setEnabled(True)  
        
        drawGraph = False
        self.matplotlibwidget_IR.axis.clear()
        for idx in range(self.idxChartPixel):            
            if ((self.inspectPixels[idx][7]>=0) and (self.inspectPixels[idx][7]<widthImageIR) and (self.inspectPixels[idx][8]>=0) and (self.inspectPixels[idx][8]<heightImageIR)):
                 
                x_values = np.linspace(self.bandsF17[0], self.bandsF17[-1], self.bandsF17.shape[0])
                if (self.inspectPixels[idx][9]==0):
                    y_values = self.imHyperIR_Pre.getPixelBands(self.inspectPixels[idx][7]-1,self.inspectPixels[idx][8]-1)
                if (self.inspectPixels[idx][9]==1):
                    y_values = self.imHyperIR.getPixelBands(self.inspectPixels[idx][7]-1,self.inspectPixels[idx][8]-1)
                if (self.inspectPixels[idx][9]==2):
                    y_values = self.imHyperIR_Post.getPixelBands(self.inspectPixels[idx][7]-1,self.inspectPixels[idx][8]-1)
                self.matplotlibwidget_IR.axis.plot(x_values,y_values,color=self.inspectPixels[idx][2])
                drawGraph = True

        
        for idxColor in range(self.idxChartBlob):
            if(self.inspectBlobs[idxColor][1] >0):
                x_values = np.linspace(self.bandsF17[0], self.bandsF17[-1], self.bandsF17.shape[0])
                self.matplotlibwidget_IR.axis.fill_between(x_values, self.spectrum_IR_lower_00[idxColor], self.spectrum_IR_upper_99[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_IR.axis.fill_between(x_values, self.spectrum_IR_lower_11[idxColor], self.spectrum_IR_upper_88[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_IR.axis.fill_between(x_values, self.spectrum_IR_lower_22[idxColor], self.spectrum_IR_upper_77[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_IR.axis.fill_between(x_values, self.spectrum_IR_lower_33[idxColor], self.spectrum_IR_upper_66[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_IR.axis.fill_between(x_values, self.spectrum_IR_lower_44[idxColor], self.spectrum_IR_upper_55[idxColor], alpha=0.15,color=self.inspectBlobs[idxColor][2])
                self.matplotlibwidget_IR.axis.plot(x_values, self.spectrum_IR_mean[idxColor],color=self.inspectBlobs[idxColor][2])                        
                self.pb_resetGraph.setEnabled(True)  
                drawGraph = True
                
        if (drawGraph):
            self.matplotlibwidget_IR.axis.plot(x_values,[0]*self.imHyperIR.getNumberBands(), linewidth=1,color='k')# JRM
            self.matplotlibwidget_IR.axis.axvline(x=self.bandsF17[self.horSlider_IR_band1.value()-1])
            self.matplotlibwidget_IR.canvas.draw()             
            self.pb_resetGraph.setEnabled(True)              
        self.matplotlibwidget_IR.canvas.draw() 
        
         #-- Write CSV file -------------------------------------------------------------------------
         #  JRM: això ha d'anar a inspect folderm, no a _ShowImages()
        
        if (self.checkedBackground and self.saveSeriesBlobs):
            fileOutput = 'Patterns\\blobs_Data_'+self.namePattern.text()+'.csv'
            with open(fileOutput, 'a', encoding='UTF8', newline='') as f1:
                writer1 = csv.writer(f1, delimiter='\t', lineterminator='\n')
            
                lastsSeriesHSCIR_x = []
                for idx, match in enumerate(self.labelsHSCIR_Match):
                    print("match", match)
                    
                    for idxImg in range(3):
                        if idxImg==0 and self.imHyperIR_Pre != -1:                    
                            #idxBlob = self.imgLabelHSC_pre == match[0]
                            idxBlob =np.argwhere(self.imgLabelIR_pre==match[1]) 
                            #print("idxBlob_pre: ",len(idxBlob))
                            for n, coord in enumerate(idxBlob):
                                y_values = self.imHyperIR_Pre.getPixelBands(coord[1],coord[0])
                                y_values = np.insert(y_values, 0, coord[1], axis=0)
                                y_values = np.insert(y_values, 0, coord[0], axis=0)
                                y_values = np.insert(y_values, 0, match[1], axis=0)
                                y_values = np.insert(y_values, 0, self.idxImgs, axis=0)
                                writer1.writerow(y_values)

                        if idxImg==1 and self.imHyperIR != -1:                    
                            #idxBlob = self.imgLabelHSC_pre == match[0]
                            idxBlob =np.argwhere(self.imgLabelIR==match[1]) 
                            #print("idxBlob: ",len(idxBlob))
                            for n, coord in enumerate(idxBlob):
                                y_values = self.imHyperIR.getPixelBands(coord[1],coord[0])
                                y_values = np.insert(y_values, 0, coord[1], axis=0)
                                y_values = np.insert(y_values, 0, coord[0]+32, axis=0)
                                y_values = np.insert(y_values, 0, match[1], axis=0)
                                y_values = np.insert(y_values, 0, self.idxImgs, axis=0)
                                writer1.writerow(y_values)
                            
                        if idxImg==2 and self.imHyperIR_Post != -1: 
                            idxBlob =np.argwhere(self.imgLabelIR_post==match[1]) 
                            #print("idxBlob_post: ",len(idxBlob))
                            for n, coord in enumerate(idxBlob):
                                y_values = self.imHyperIR_Post.getPixelBands(coord[1],coord[0])
                                y_values = np.insert(y_values, 0, coord[1], axis=0)
                                y_values = np.insert(y_values, 0, coord[0]+64, axis=0)
                                y_values = np.insert(y_values, 0, match[1], axis=0)
                                y_values = np.insert(y_values, 0, self.idxImgs, axis=0)
                                writer1.writerow(y_values)
                        


    #== _readHypercubes ==============================================================================
    #== A partir del index (que s'obté amb la funció )  
    #== Also shows Pre and Post images
    #---- Input --------------------------------------------------------------------------------------
    # - idx: 
    # - idxPrevious:
    #---- Output -------------------------------------------------------------------------------------
    #=================================================================================================
    def _readHypercubes(self,idx, idxPrevious):
        
        self.img_Color_pre = []
        self.img_Color = []
        self.img_Color_post = []
        
        # Show the name of the selected image (color file only)
        # Enable or disable buttons to access the previous or next image.
        # Shows the index of the inspected image (along with the total number of images)
        if (idx == 0):
            self.lab_PreImage.setText('---')
            self.pButt_previousImage.setEnabled(False)            
        else:
            self.lab_PreImage.setText(self.listImgs[idx-1][0])            
            self.pButt_previousImage.setEnabled(True)    
            
        self.lab_Image.setText(self.listImgs[idx][0])
        
        if ((idx+1) == self.totalImgs):
            self.lab_PostImage.setText('---')
            self.pButt_nextImage.setEnabled(False)
        else:
            self.lab_PostImage.setText(self.listImgs[idx+1][0])        
            self.pButt_nextImage.setEnabled(True)        
            
        self.lab_InspImages.setText(str(idx+1)+' / '+str(self.totalImgs))        
        
        # -- Obte les imatges COLOR del index especificat --------------------------------------------------
        if (self.totalImgs>0):
            color_File = os.path.join(self.folderpath,self.listImgs[idx][0])
            self.img_Color = cv2.imread(color_File)
            
            h, w, c = self.img_Color.shape
            self.lab_Color_width.setEnabled(True)
            self.lab_Color_width.setText(str(w))
            self.lab_Color_Height.setEnabled(True)
            self.lab_Color_Height.setText(str(h))
            self.lab_Color_Bands.setEnabled(True)
            self.lab_Color_Bands.setText(str(c))   
            self.label_1_color.setEnabled(True)
            self.label_2_color.setEnabled(True)
            self.label_3_color.setEnabled(True)
        else:
            self.img_Color = []
            self.lab_Color_width.setEnabled(False)
            self.lab_Color_width.setText('-----')
            self.lab_Color_Height.setEnabled(False)
            self.lab_Color_Height.setText('-----')
            self.lab_Color_Bands.setEnabled(False)
            self.lab_Color_Bands.setText('-----')   
            self.label_1_color.setEnabled(False)
            self.label_2_color.setEnabled(False)
            self.label_3_color.setEnabled(False)            
        
        if (idx > 0):
            color_File_pre = os.path.join(self.folderpath,self.listImgs[idx-1][0])
            self.img_Color_pre = cv2.imread(color_File_pre)
        else:
            self.img_Color_pre = []
        
        
        if ((idx+1) < self.totalImgs):
            color_File_post = os.path.join(self.folderpath,self.listImgs[idx+1][0])
            self.img_Color_post = cv2.imread(color_File_post)
        else:
            self.img_Color_post = []
            
        # -- Obte els hipercubs HSC del index especificat --------------------------------------------------
        if (idx == idxPrevious-1): # selected previous (it saves us regenerating the HSC cube)
            self.imHyperHSC_Post = self.imHyperHSC
            self.imHyperHSC = self.imHyperHSC_Pre
            if(idx>0):
                hsc_File = os.path.join(self.folderpath,self.listImgs[idx-1][1])
                self.imHyperHSC_Pre = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            else:
                self.imHyperHSC_Pre = -1
        elif (idx == idxPrevious+1): # selected next (it saves us regenerating the HSC cube)
            self.imHyperHSC_Pre =self.imHyperHSC
            self.imHyperHSC = self.imHyperHSC_Post
            if((idx+1) < self.totalImgs):
                hsc_File = os.path.join(self.folderpath,self.listImgs[idx+1][1])
                self.imHyperHSC_Post = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            else:
                self.imHyperHSC_Post = -1
        else:
            hsc_File = os.path.join(self.folderpath,self.listImgs[idx][1])
            self.imHyperHSC = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)        
            if(idx>0):
                hsc_File = os.path.join(self.folderpath,self.listImgs[idx-1][1])
                self.imHyperHSC_Pre = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            else:
                self.imHyperHSC_Pre = -1
            if((idx+1) < self.totalImgs):
                hsc_File = os.path.join(self.folderpath,self.listImgs[idx+1][1])
                self.imHyperHSC_Post = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            else:
                self.imHyperHSC_Post = -1

        # -- Obte els hipercubs IR del index especificat --------------------------------------------------
        if (idx == idxPrevious-1): # selected previous (it saves us regenerating the IR cube)
            self.imHyperIR_Post = self.imHyperIR
            self.imHyperIR = self.imHyperIR_Pre
            if(idx>0):
                ir_File = os.path.join(self.folderpath,self.listImgs[idx-1][2])
                self.imHyperIR_Pre = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            else:
                self.imHyperIR_Pre = -1
        elif (idx == idxPrevious+1): # selected next (it saves us regenerating the IR cube)
            self.imHyperIR_Pre =self.imHyperIR
            self.imHyperIR = self.imHyperIR_Post
            if((idx+1) < self.totalImgs ):
                ir_File = os.path.join(self.folderpath,self.listImgs[idx+1][2])
                self.imHyperIR_Post = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            else:
                self.imHyperIR_Post = -1
        else:
            ir_File = os.path.join(self.folderpath,self.listImgs[idx][2])
            self.imHyperIR = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)        
            if(idx>0):
                ir_File = os.path.join(self.folderpath,self.listImgs[idx-1][2])
                self.imHyperIR_Pre = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            else:
                self.imHyperIR_Pre = -1
            if((idx+1) < self.totalImgs):
                ir_File = os.path.join(self.folderpath,self.listImgs[idx+1][2])
                self.imHyperIR_Post = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            else:
                self.imHyperIR_Post = -1
                
        # Extreu els binaris i els labels
        if (self.checkedBackground):
            imgBinHSC_pre = []
            imgBinHSC = []
            imgBinHSC_post = []
            imgBinIR_pre = []
            imgBinIR = []
            imgBinIR_post = []
            
            if(self.imHyperHSC_Pre != -1):
                imgBinHSC_pre = self._extractBackground(self.imHyperHSC_Pre,1)
            if(self.imHyperHSC != -1):
                imgBinHSC = self._extractBackground(self.imHyperHSC,1)
            if(self.imHyperHSC_Post != -1):
                imgBinHSC_post = self._extractBackground(self.imHyperHSC_Post,1)
            if(self.imHyperIR_Pre != -1):
                imgBinIR_pre = self._extractBackground(self.imHyperIR_Pre,2)
            if(self.imHyperIR != -1):
                imgBinIR = self._extractBackground(self.imHyperIR,2)
            if(self.imHyperIR_Post != -1):
                imgBinIR_post = self._extractBackground(self.imHyperIR_Post,2)
                       
            self.labelsHSCIR_Match, self.labelsHSC_Only, self.labelsIR_Only, self.imgLabelHSC_pre, self.imgLabelHSC, self.imgLabelHSC_post, self.imgLabelIR_pre, self.imgLabelIR,self.imgLabelIR_post, imgAnalized = self.matchLabels(imgBinHSC_pre,imgBinHSC,imgBinHSC_post,imgBinIR_pre,imgBinIR,imgBinIR_post,self._OffX_HSCvsIR,self._OffY_HSCvsIR)

    def selectedPatternInList(self):
        self.idxPatterns = self.listPatterns.currentRow()
        self.pButt_DeletePattern.setEnabled(True)
        #self.resetGraph()
        #self.idxImgsPrevious = self.idxImgs
        #self.idxImgs = self.listWidget.currentRow()
        #self._readHypercubes(self.idxImgs,self.idxImgsPrevious)
        #self._ShowImages()        
    
    def selectedInImageInList(self):
        self.resetGraph()
        self.idxImgsPrevious = self.idxImgs
        self.idxImgs = self.listWidget.currentRow()
        self._readHypercubes(self.idxImgs,self.idxImgsPrevious)
        self._ShowImages() 
        
    def selectedPreviousImage(self):

        self.resetGraph()
        self.idxImgsPrevious = self.idxImgs
        self.idxImgs -= 1;
        self.listWidget.setCurrentRow(self.idxImgs)
        self.listWidget.setFocus() 
        
        self.pButt_nextImage.setEnabled(True)
        if (self.idxImgs == 0):
            self.pButt_previousImage.setEnabled(False) 
        self._readHypercubes(self.idxImgs,self.idxImgsPrevious)
        self._ShowImages()        
        
    def selectedNextImage(self):
        
        self.resetGraph()
        self.idxImgsPrevious = self.idxImgs
        self.idxImgs += 1;
        self.listWidget.setCurrentRow(self.idxImgs)
        self.listWidget.setFocus() 
        
        self.pButt_previousImage.setEnabled(True)
        if ((self.idxImgs+1) == self.totalImgs):
            self.pButt_nextImage.setEnabled(False)       
        self._readHypercubes(self.idxImgs,self.idxImgsPrevious)
        self._ShowImages()        
    
    def deletePattern(self):
        listItems=self.listPatterns.selectedItems()
        if not listItems: return        
        
        for item in listItems:
            print(self.listPatterns.currentItem().text())
            file=".\\Patterns\\"+self.listPatterns.currentItem().text()+".csv"
            os.remove(file)
            print(file)
            self.listPatterns.takeItem(self.listPatterns.row(item))
            
        self.listPatterns.setCurrentRow( -1)
        self.pButt_DeletePattern.setEnabled(False)
    
    def batchInspection(self):
        self.resetGraph()

        self.lastsSeriesHSCIR_0= []   #  Conté les 2 últimes inspeccions realitzades
        self.lastsSeriesHSCIR_1= []  #  Conté les 2 últimes inspeccions realitzades
        

        for idx in range(self.totalImgs): 
            self.idxImgs = idx #JRM
            if (idx==0):
                self._readHypercubes(idx,-2)
            else:
                self._readHypercubes(idx,idx-1)
            
            self._ShowImages()        
           # self.labColor.repaint()  #Força que s'actualizi el form
            self.repaint()
            self.update()                       
            self.show()
            #self.repaint()
            
            self.labColor_pre.repaint()
            self.labColor.repaint()
            self.labColor_post.repaint()
            
            
            #widget.update() ##JRM2
            #app.update()    ##JRM2
            #app.repaint()

            
        self.idxImgs = self.totalImgs-1
        self.listWidget.setCurrentRow(self.totalImgs-1)
        self.listWidget.setFocus() 
        
    def learningPattern(self):
        print("JRM: Entra al la funcio 'learningPattern'")
        self.resetGraph()
        
        strName= "".join(self.namePattern.text().split())
        self.namePattern.setText(strName)
        
        if len(strName) == 0:
            self.namePattern.setFocus()
        else:            
            self.saveSeriesBlobs = True
            self.idxPixel_findPattern = 0
            self.idxImage_findPattern = 0
            print("JRM: Entra al la funcio 'learningPattern' i crida --> batchInspection()")
            self.batchInspection()
            self.saveSeriesBlobs = False
            
            self.lab_AnalyzingPattern.show()
            self.lab_AnalyzingPattern.repaint()
            dfBlobs = hp.extractDescriptorSignal(r"C:\\Users\jrosell\Hyperspectral\Patterns\blobs_Data_HDPE2.csv")
            dfPatt = hp.extractPatternsAllBlobs(dfBlobs,"HDPE")
            dfPatt.to_csv(r"C:\\Users\jrosell\Hyperspectral\Patterns\Pattern_HDPE2.csv")
                      
            self.lab_AnalyzingPattern.hide()  
        

    ##################################################################################################
    ###  Funtions inspect Background 
    ##################################################################################################
        
    def learningBackground(self): 
#        print('Call: learningBackground()')

        self.resetGraph()
        
        imgHSC_Width = int(self.imHyperHSC.getWidth())
        imgHSC_Height = self.imHyperHSC.getHeight()
        imgHSC_Bands = self.imHyperHSC.getNumberBands()        

        imgIR_Width = self.imHyperIR.getWidth()
        imgIR_Height = self.imHyperIR.getHeight()
        imgIR_Bands = self.imHyperIR.getNumberBands()        
        
        # initialization of imgsHSC_BackGround_Max structures
        self.imgHyperHSC_BackGround_Max =[] 
        for idx in range(imgHSC_Bands):
            img = np.zeros((imgHSC_Height,imgHSC_Width,1), np.int16)
            self.imgHyperHSC_BackGround_Max.append(np.array(img).reshape((imgHSC_Height,imgHSC_Width)))

        self.imgHyperIR_BackGround_Max =[] 
        for idx in range(imgIR_Bands):
            img = np.zeros((imgIR_Height,imgIR_Width,1), np.int16)
            self.imgHyperIR_BackGround_Max.append(np.array(img).reshape((imgIR_Height,imgIR_Width)))
            
        # initialization of imgsHSC_BackGround_Min structures
        self.imgHyperHSC_BackGround_Min =[]
        for idx in range(imgHSC_Bands):
            img = np.ones((imgHSC_Height,imgHSC_Width,1), np.int16)*9999
            self.imgHyperHSC_BackGround_Min.append(np.array(img).reshape((imgHSC_Height,imgHSC_Width)))

        self.imgHyperIR_BackGround_Min =[]
        for idx in range(imgIR_Bands):
            img = np.ones((imgIR_Height,imgIR_Width,1), np.int16)*9999
            self.imgHyperIR_BackGround_Min.append(np.array(img).reshape((imgIR_Height,imgIR_Width)))
            
       
        # Inspect the set of background images
        for idxImg in range(self.totalImgs): 
            if (idxImg==0):
                self._readHypercubes(idxImg,-2)
            else:
                self._readHypercubes(idxImg,idxImg-1)
            self._ShowImages()
            self.labColor.repaint()  #Força que s'actualizi el form
            
            hsc_File = os.path.join(self.folderpath,self.listImgs[idxImg][1])
            self.imHyperHSC = HypSpImage(hsc_File,self._HSC_WIDTH,self._HSC_HEIGHT,self.F10FilePath)
            for idx in range(imgHSC_Bands):
                cvImg = self.imHyperHSC.getBWimage(idx=idx)
                imgINT = cvImg.astype(np.int16)

                imgSub = imgINT - self.imgHyperHSC_BackGround_Max[idx]
                idxsNeg = (imgSub > 0)
                self.imgHyperHSC_BackGround_Max[idx][idxsNeg] = imgINT[idxsNeg]

                imgSub = imgINT - self.imgHyperHSC_BackGround_Min[idx]
                idxsNeg = (imgSub < 0)
                self.imgHyperHSC_BackGround_Min[idx][idxsNeg] = imgINT[idxsNeg]

            ir_File = os.path.join(self.folderpath,self.listImgs[idxImg][2])
            self.imHyperIR = HypSpImage(ir_File,self._IR_WIDTH,self._IR_HEIGHT,self.F17FilePath)
            for idx in range(imgIR_Bands):
                cvImg = self.imHyperIR.getBWimage(idx=idx)
                imgINT = cvImg.astype(np.int16)

                imgSub = imgINT - self.imgHyperIR_BackGround_Max[idx]
                idxsNeg = (imgSub > 0)
                self.imgHyperIR_BackGround_Max[idx][idxsNeg] = imgINT[idxsNeg]

                imgSub = imgINT - self.imgHyperIR_BackGround_Min[idx]
                idxsNeg = (imgSub < 0)
                self.imgHyperIR_BackGround_Min[idx][idxsNeg] = imgINT[idxsNeg]

        num_rows, num_cols = self.imgHyperHSC_BackGround_Min[0].shape
        for idx in range(imgHSC_Bands):
            for idxCol in range(num_cols):
                minVal = np.min(self.imgHyperHSC_BackGround_Min[idx][:,idxCol])
                self.imgHyperHSC_BackGround_Min[idx][:,idxCol] = minVal
                maxVal = np.max(self.imgHyperHSC_BackGround_Max[idx][:,idxCol])
                self.imgHyperHSC_BackGround_Max[idx][:,idxCol] = maxVal
                       
        num_rows, num_cols = self.imgHyperIR_BackGround_Min[0].shape
        for idx in range(imgIR_Bands):
            for idxCol in range(num_cols):
                minVal = np.min(self.imgHyperIR_BackGround_Min[idx][:,idxCol])
                self.imgHyperIR_BackGround_Min[idx][:,idxCol] = minVal
                maxVal = np.max(self.imgHyperIR_BackGround_Max[idx][:,idxCol])
                self.imgHyperIR_BackGround_Max[idx][:,idxCol] = maxVal
            
                
        for idx in range(imgHSC_Bands):            
            diff = ((self.imgHyperHSC_BackGround_Max[idx]-self.imgHyperHSC_BackGround_Min[idx]).astype(float)*0.20).astype(np.int16)
            self.imgHyperHSC_BackGround_Max[idx] += diff
            self.imgHyperHSC_BackGround_Min[idx] -= diff
            self.imgHyperHSC_BackGround_Max[idx] += 20
            self.imgHyperHSC_BackGround_Min[idx] -= 20
            idxsNeg = (self.imgHyperHSC_BackGround_Min[idx] < 0)
            self.imgHyperHSC_BackGround_Min[idx][idxsNeg] = 0

        for idx in range(imgIR_Bands):            
            diff = ((self.imgHyperIR_BackGround_Max[idx]-self.imgHyperIR_BackGround_Min[idx]).astype(float)*0.20).astype(np.int16)
            self.imgHyperIR_BackGround_Max[idx] += diff
            self.imgHyperIR_BackGround_Min[idx] -= diff
            self.imgHyperIR_BackGround_Max[idx] += 25
            self.imgHyperIR_BackGround_Min[idx] -= 25
            idxsNeg = (self.imgHyperIR_BackGround_Min[idx] < 0)
            self.imgHyperIR_BackGround_Min[idx][idxsNeg] = 0
            
            
        self.idxImgs = self.totalImgs-1
        self.listWidget.setCurrentRow(self.totalImgs-1)
        self.listWidget.setFocus()           
    
            
        if (self.totalImgs>=10):
            self.rb_Segm_Not.setEnabled(True)
            self.rb_Segm_Bin.setEnabled(True)
            self.rb_Segm_Mark.setEnabled(True)
            self.rb_graph_SelectPixel.setEnabled(True)
            self.rb_graph_SelectBlob.setEnabled(True)
            self.cb_recordSpectrums.setEnabled(True)            
            self.checkedBackground = True
            self.lab_RequiresMessage_2.setHidden(False)
            self.pButt_learningPattern.setEnabled(True)  
            self.namePattern.setEnabled(True)
            
        else:
            self.rb_Segm_Not.setEnabled(False)
            self.rb_Segm_Bin.setEnabled(False)
            self.rb_Segm_Mark.setEnabled(False)
            self.rb_graph_SelectPixel.setEnabled(False)
            self.rb_graph_SelectBlob.setEnabled(False)
            self.cb_recordSpectrums.setEnabled(False)
            self.comboBox_plasticType.setEnabled(False)
            self.checkedBackground = False

    def markBoundingBox(self):
        self._ShowImages()
        
    def markBackground(self):
        self._ShowImages()
        
    def markBlobs(self):
        self._ShowImages()
        
    def doNotSegment(self):
        if self.rb_Segm_Not.isChecked():
            self.cb_MarkBG.setEnabled(False)
            self.cb_MarkBlobs.setEnabled(False)            
            self.cb_MarkBoundingBox.setEnabled(False)     
        self._ShowImages()
        
    def selectBinary(self):
        if self.rb_Segm_Bin.isChecked():
            self.cb_MarkBG.setEnabled(False)
            self.cb_MarkBlobs.setEnabled(False)     
            self.cb_MarkBoundingBox.setEnabled(False)     
        self._ShowImages()
        
    def selectMark(self):
        if self.rb_Segm_Mark.isChecked():
            self.cb_MarkBG.setEnabled(True)
            self.cb_MarkBlobs.setEnabled(True)            
            self.cb_MarkBoundingBox.setEnabled(True)     
        self._ShowImages()
        
    def selectPixel(self):
        #self.rb_graph_SelectBlob.setEnabled(True)
        pass

    def selectBlob(self):
        #self.rb_graph_SelectPixel.setEnabled(True)
        pass
    
    def recordSpectrumsInFile(self):
        if (self.cb_recordSpectrums.isChecked() and self.checkedBackground):
            self.comboBox_plasticType.setEnabled(True)
        else:
            self.comboBox_plasticType.setEnabled(False)
        pass
   
    ##################################################################################################
    ###  Auxiliar funtions 
    ##################################################################################################
    def extractBlobsValues(imgBinColHSC, imgBinColIR):
        pass

        
if __name__ == '__main__':
    app = QApplication(sys.argv)
    GUI = Hyperspectral_GUI()
    GUI.show()
    sys.exit(app.exec_())
    

    

In [ ]:
import glob, os
for file in glob.glob(r"C:\Users\jrosell\Hyperspectral\Patterns\*.csv"):
    print(file)

In [2]:
#Comparativa de temps d'inserció de dades en una estructura
from datetime import datetime
import numpy as np

tstamp1 = datetime.now()
arr = np.empty((0,3), int)
for x in range(100000):
    arr = np.append(arr, np.array([[x,x+1,x+2]]), axis=0)
tstamp2 = datetime.now()
td = tstamp2 - tstamp1
print("array -----> ",td.total_seconds(), "s")


tstamp1 = datetime.now()
arr = []
for x in range(100000):
    arr.append([x,x+1,x+2])
tstamp2 = datetime.now()
td = tstamp2 - tstamp1
print("np.array --> ",td.total_seconds(), "s")



array ----->  5.254377 s
np.array -->  0.054951 s
